In [1]:
%run ontologyPackage/ontologySTATanalysis.ipynb

In [2]:
import pandas as pd 
import numpy as np
from numpy import random, array
import time 
from multiprocessing import Pool
import rpy2.robjects as robjects

import os
from goatools.obo_parser import GODag

pd.set_option('display.max_colwidth', -1) # Values in columns won't be shortned 
pd.set_option('chained_assignment',None) # Disabling chained assignments 

geneColID = ["chromosome","source","type","start","end","score","strand","phase","gene_symbol","gene_ensID","length","entrezid"]
geneAnnotationDF = pd.read_csv('entrez_id/geneAnnotationsDF_Selected_entrezID.csv', sep=',', comment='#', low_memory=False, header=0, names=geneColID)
chromosomeColID = ['chromosome','source','type','start','end','score','strand','phase']
chromosomesDF = pd.read_csv('chromosomesDF.csv', sep=',', comment='#', low_memory=False, header=0, names=chromosomeColID)

In [3]:
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests as padjust

In [4]:
# GO Term information
if not os.path.exists('go-basic.obo'):
    !wget http://geneontology.org/ontology/go-basic.obo
goDATA = GODag('go-basic.obo', optional_attrs=['relationship'])

go-basic.obo: fmt(1.2) rel(2019-07-01) 47,413 GO Terms; optional_attrs(relationship)


In [5]:
#Dropping unneccesary stuff and resetting index from 0
# geneAnnotationDF = geneAnnotationDF[geneAnnotationDF.gene_symbol != 'CTD-2207O23.3']
geneAnnotationDF = geneAnnotationDF.sort_values(by=['chromosome', 'start'])
chromosomesDF = chromosomesDF.sort_values(by=['chromosome'])
chromosomesDF = chromosomesDF.reset_index()
geneAnnotationDF = geneAnnotationDF.reset_index()
chromosomesDF = chromosomesDF.drop(columns=['score', 'strand', 'phase', 'index'])
geneAnnotationDF = geneAnnotationDF.drop(columns=['score', 'phase', 'index'])

In [6]:
# Making a list of DataFrames to be used in addwindow tss so that if - gene the start is its end and its end is its start 
u = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y'] # Chromosome ids 
geneDFL = []
for c in u:
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == c].copy()
    for i, row in geneDF.iterrows():
        if row['strand'] == '-':
            start = row['end']
            end = row['start']
            geneDF.at[i, 'start'] = start
            geneDF.at[i, 'end'] = end
    geneDF = geneDF.sort_values(by=['start'])
    geneDF = geneDF.reset_index()
    geneDF = geneDF.drop(columns=['index'])
    geneDFL.append(geneDF)

In [7]:
# Making a list of DataFrames to be used in future functions in var geneDFL
u = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y']
geneDFLBODY = []
for cc in u:
    geneDFbod = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == cc].copy()
    geneDFbod = geneDFbod.sort_values(by=['start'])
    geneDFbod = geneDFbod.reset_index()
    geneDFbod = geneDFbod.drop(columns=['index'])
    geneDFLBODY.append(geneDFbod)

In [8]:
# probability that a randomly chosen gene is in a certain chromosome. Ordered from 1 - MT,X,Y | As shown chromosome 1 has highest prob of selection
probabilityL = [0.08069467597786323, 0.07850260775517634, 0.06427388269957329, 0.06165457287524136, 0.05884231002379223, 0.05536363751707386, 0.05164908592141782, 0.0470440371698401, 0.044858119022639725, 0.043367989830914035, 0.043785860460049814, 0.04319875643982657, 0.037069107410936775, 0.034696265410969616, 0.033058580434273406, 0.029281523923645837, 0.026986378244674356, 0.026051531764496164, 0.018999829086634144, 0.02088839915494138, 0.015140187376094325, 0.016471877735809576, 5.370214538878023e-06, 0.05057780526426391, 0.017537607961181506]

In [9]:
# Generates random sites
dl = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25'] # 23, 24, 25
chrD = {'1': [1, 248956422],'2': [1, 242193529],'3': [1, 198295559],'4': [1, 190214555],'5': [1, 181538259],'6': [1, 170805979],'7': [1, 159345973],'8': [1, 145138636],'9': [1, 138394717],'10': [1, 133797422],'11': [1, 135086622],'12': [1, 133275309],'13': [1, 114364328],'14': [1, 107043718],'15': [1, 101991189],'16': [1, 90338345],'17': [1, 83257441],'18': [1, 80373285],'19': [1, 58617616],'20': [1, 64444167],'21': [1, 46709983],'22': [1, 50818468],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]}
def nRandSites(data):
    nSite = data[0]
    rand = data[1] 
    sitesbyC =[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

    random.seed(rand)
        
    for site in range(nSite):
        chrN = random.choice(dl, p=probabilityL)
        randnum = round(random.uniform(chrD[chrN][0], chrD[chrN][1]))
        sitesbyC[int(chrN) - 1].append(randnum)
    for l in sitesbyC:
        l.sort()
    return array([array(l) for l in sitesbyC])

def nRandSitesSim(nSite, nSim):
    totalsites = [nSite] * nSim
    chunks = []
    for i in totalsites:
        chunks.append([i])
    for l in chunks:
        l.append(random.randint(100000))
    
    pool = Pool(5)
    result = pool.map(nRandSites, chunks)
    pool.close()
    return(result) 

In [10]:
##### Adds midpoint in gene and upper and lower bound values to geneDFLtss based on size of window or nearby genes. Adding around geneTSS.
def addWindowTSS(window):
    chrD = {'1': [1, 248956422],'2': [1, 242193529],'3': [1, 198295559],'4': [1, 190214555],'5': [1, 181538259],'6': [1, 170805979],'7': [1, 159345973],'8': [1, 145138636],'9': [1, 138394717],'10': [1, 133797422],'11': [1, 135086622],'12': [1, 133275309],'13': [1, 114364328],'14': [1, 107043718],'15': [1, 101991189],'16': [1, 90338345],'17': [1, 83257441],'18': [1, 80373285],'19': [1, 58617616],'20': [1, 64444167],'21': [1, 46709983],'22': [1, 50818468],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]} 
    geneWindowDFL = []
    for i in range(len(geneDFL)):
        df = geneDFL[i].copy()
        geneWindowDFL.append(df)
    outList = []
    chrIDNUM = 0
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        chrIDNUM += 1 
        dfIndexMax = len(dfG) - 1   
        for i, row in dfG.iterrows():
            start = row['start']
            
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + start) # Middle of gene body 
            if i == 0 or i == dfIndexMax: # Case for start and end genes
                if i == 0:
                    dfG.at[i, 'lowB'] = max(start - window, chrD[str(chrIDNUM)][0]) # chrD[chrIDNUM][0] = start val of the chromosome
                    dfG.at[i, 'upperB'] = min(start + window, ((dfG.iat[i+1, 3] - start) / 2) + start)
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 3]) / 2)) # dfG.iat[i-1, 3] = start of gene below
                    dfG.at[i, 'upperB'] = min(start + window, chrD[str(chrIDNUM)][1]) # chrD[chrIDNUM][1] = end val of chromosome 
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 3]) / 2)) # dfG.iat[i-1, 3] = start of gene below
                dfG.at[i, 'upperB'] = min(start + window, ((dfG.iat[i+1, 3] - start) / 2) + start) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)
#         dfG['domainLEN'] = dfG['upperB'] - dfG['lowB']
        LBandUB = []
        LBandUB.append(dfG.lowB.values)
        LBandUB.append(dfG.upperB.values)
        outList.append(LBandUB)
    
    return(outList)

In [11]:
##### Adds midpoint in gene and upper and lower bound values to geneDFLtss based on size of window or nearby genes. Adding around geneTSS.
def addWindowTSS20(window):
    chrD = {'1': [1, 248956422],'2': [1, 242193529],'3': [1, 198295559],'4': [1, 190214555],'5': [1, 181538259],'6': [1, 170805979],'7': [1, 159345973],'8': [1, 145138636],'9': [1, 138394717],'10': [1, 133797422],'11': [1, 135086622],'12': [1, 133275309],'13': [1, 114364328],'14': [1, 107043718],'15': [1, 101991189],'16': [1, 90338345],'17': [1, 83257441],'18': [1, 80373285],'19': [1, 58617616],'20': [1, 64444167],'21': [1, 46709983],'22': [1, 50818468],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]} 
    geneWindowDFL = []
    for i in range(len(geneDFL)):
        df = geneDFL[i].copy()
        geneWindowDFL.append(df)
    chrIDNUM = 0
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        chrIDNUM += 1 
        dfIndexMax = len(dfG) - 1   
        for i, row in dfG.iterrows():
            start = row['start']
            
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + start) # Middle of gene body 
            if i == 0 or i == dfIndexMax: # Case for start and end genes
                if i == 0:
                    dfG.at[i, 'lowB'] = max(start - window, chrD[str(chrIDNUM)][0]) # chrD[chrIDNUM][0] = start val of the chromosome
                    dfG.at[i, 'upperB'] = min(start + window, ((dfG.iat[i+1, 3] - start) / 2) + start)
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 3]) / 2)) # dfG.iat[i-1, 3] = start of gene below
                    dfG.at[i, 'upperB'] = min(start + window, chrD[str(chrIDNUM)][1]) # chrD[chrIDNUM][1] = end val of chromosome 
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 3]) / 2)) # dfG.iat[i-1, 3] = start of gene below
                dfG.at[i, 'upperB'] = min(start + window, ((dfG.iat[i+1, 3] - start) / 2) + start) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)
        dfG['domainLEN'] = dfG['upperB'] - dfG['lowB']
    
    return(geneWindowDFL)

In [12]:
# Adds midpoint in gene and upper and lower bound values to geneDFL based on size of window or nearby genes. Adding around geneBODY.
def addWindowBODY(window):
    chrD = {'1': [1, 248956422],'2': [1, 242193529],'3': [1, 198295559],'4': [1, 190214555],'5': [1, 181538259],'6': [1, 170805979],'7': [1, 159345973],'8': [1, 145138636],'9': [1, 138394717],'10': [1, 133797422],'11': [1, 135086622],'12': [1, 133275309],'13': [1, 114364328],'14': [1, 107043718],'15': [1, 101991189],'16': [1, 90338345],'17': [1, 83257441],'18': [1, 80373285],'19': [1, 58617616],'20': [1, 64444167],'21': [1, 46709983],'22': [1, 50818468],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]} 
    chrIDNUM = 0
    geneWindowDFL = []
    for i in range(len(geneDFLBODY)):
        df = geneDFLBODY[i].copy()
        geneWindowDFL.append(df)
    
    outList = []
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        chrIDNUM += 1 
        dfIndexMax = len(dfG) - 1         
        for i, row in dfG.iterrows():
            start = row['start']
            end = row['end']
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + row['start']) # Middle of gene body 
            
            if i == 0 or i == dfIndexMax: # Case for start and end sites
                if i == 0:
                    dfG.at[i, 'lowB'] = max(start - window, chrD[str(chrIDNUM)][0]) # end of the gene below position
                    dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end)
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(start - window, start - (abs(start - dfG.iat[i-1, 4]) / 2)) 
                    dfG.at[i, 'upperB'] = min(row['end'] + window, chrD[str(chrIDNUM)][1])
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(start - window, start - (abs(start - dfG.iat[i-1, 4]) / 2)) # end of the gene below position [i,4]
                dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)
#         dfG['domainLEN'] = dfG['upperB'] - dfG['lowB']
        LBandUB = []
        LBandUB.append(dfG.lowB.values)
        LBandUB.append(dfG.upperB.values)
        outList.append(LBandUB)
    
    return(outList)

In [13]:
# Adds midpoint in gene and upper and lower bound values to geneDFL based on size of window or nearby genes. Adding around geneBODY.
def addWindowBODY20(window):
    chrD = {'1': [1, 248956422],'2': [1, 242193529],'3': [1, 198295559],'4': [1, 190214555],'5': [1, 181538259],'6': [1, 170805979],'7': [1, 159345973],'8': [1, 145138636],'9': [1, 138394717],'10': [1, 133797422],'11': [1, 135086622],'12': [1, 133275309],'13': [1, 114364328],'14': [1, 107043718],'15': [1, 101991189],'16': [1, 90338345],'17': [1, 83257441],'18': [1, 80373285],'19': [1, 58617616],'20': [1, 64444167],'21': [1, 46709983],'22': [1, 50818468],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]} 
    chrIDNUM = 0
    geneWindowDFL = []
    for i in range(len(geneDFLBODY)):
        df = geneDFLBODY[i].copy()
        geneWindowDFL.append(df)
    
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        chrIDNUM += 1 
        dfIndexMax = len(dfG) - 1         
        for i, row in dfG.iterrows():
            start = row['start']
            end = row['end']
            
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + row['start']) # Middle of gene body 
            
            if i == 0 or i == dfIndexMax: # Case for start and end sites
                if i == 0:
                    dfG.at[i, 'lowB'] = max(start - window, chrD[str(chrIDNUM)][0]) # end of the gene below position
                    dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end)
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(start - window, start - (abs(start - dfG.iat[i-1, 4]) / 2)) 
                    dfG.at[i, 'upperB'] = min(row['end'] + window, chrD[str(chrIDNUM)][1])
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(start - window, start - (abs(start - dfG.iat[i-1, 4]) / 2)) # end of the gene below position [i,4]
                dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)
        dfG['domainLEN'] = dfG['upperB'] - dfG['lowB']
    
    return(geneWindowDFL)

In [14]:
def findSMLdomain(windowDF):
    windowLens = windowDF
    allVALS = []
    for df in windowLens:
        values = list(df.domainLEN.values)
        allVALS += values
    allVALS = sorted(allVALS)
    cutoff = int(len(allVALS) / 3)
    small = allVALS[:cutoff][-1]
    medium = allVALS[cutoff:cutoff*2][-1]
    return [small, medium]

In [15]:
# Creating list of all the entrezids 
entrezIDLtss = []
for geneDF in geneDFL:
    entrezIDLtss.append(geneDF.entrezid.values)

In [16]:
entrezIDLbody = []
for geneDF in geneDFLBODY:
    entrezIDLbody.append(geneDF.entrezid.values)

In [17]:
# CLOSEST Genes potentionally regulated by sites in window with respect to the genes body. k = 1 
def geneReadSites(bsL, geneWindow, method='TSS'): # geneWindow = windows lists
    #     geneWindow = addWindowBODY(window)
    chromosomeI = 0 # 0 == chromosome group 1          
    entrezidOutL = [] # Output list of entrez ids 
    if method == 'TSS':
        entrezIDL = entrezIDLtss
    if method == 'BODY':
        entrezIDL = entrezIDLbody
    if method != 'TSS'and method != 'BODY':
        return 'method must be TSS or BODY'
    
    for bounds in geneWindow:
        lowB = bounds[0]
        upperB = bounds[1]
        geneIDS = entrezIDL[chromosomeI]
        sitesL = bsL[chromosomeI]
        
        for site in sitesL:
            for i in range(len(lowB)):
                if site < lowB[i]:
                    lowB = lowB[i:] # Getting rid of the gene windows because it has been mapped. Genes at a lower positions are removed casuse they are small
                    upperB = upperB[i:]
                    geneIDS = geneIDS[i:]
                    break
                if lowB[i] <= site and upperB[i] > site:
                    entrezidOutL.append(geneIDS[i])
#                     lowB = lowB[i+1:] # Getting rid of the gene windows because it has been mapped. Genes at a lower positions are removed casuse they are small
#                     upperB = upperB[i+1:]
#                     geneIDS = geneIDS[i+1:]
#                     break # Done with the current site so break 
        chromosomeI += 1   
    return entrezidOutL

In [18]:
# Given a list of lists with go term and its odds-ratio,p-val return a list of just GO terms 
def getGOfromAnalysis(goAnalysis):
    goTermL = []
    for key in goAnalysis:
        goTermL.append(key)
    return goTermL

In [19]:
# Given original analysis and simulated analysis. Adds count to go term in original list if the random go term has a lower p-val and higher odds ratio 
def compareGOAnalysis(origAnalysis, counters):
    for k in counters.keys():
        if origAnalysis[k][2] != 'NA':
            if (counters[k][0] < origAnalysis[k][2]): #  (simAnalysis[k][0] > origAnalysis[k][0]) and 
                origAnalysis[k][5] += 1
        if origAnalysis[k][3] != 'NA':
            if (counters[k][1] < origAnalysis[k][3]): 
                origAnalysis[k][6] += 1
        if origAnalysis[k][4] != 'NA':
            if (counters[k][2] < origAnalysis[k][4]): 
                origAnalysis[k][7] += 1
    return origAnalysis 

In [20]:
# converts analysis to list format and divides by nSim to get refabs p value
def convertAnalysistoFormat(analysis, nSim):
    analist = []
    for k in analysis.keys():
        val = analysis[k]
        outval = val[:2]
        app = val[5:]
        pval = []
        for i in app:
            p = (i + 1) / (nSim + 1)
            pval.append(p)
        nas = val[2:5]
        for i in range(3):
            if nas[i] == 'NA':
                pval[i] = 'NA'
        outval += pval
        analist.append([k, outval])
    return analist

In [21]:
# Combines the three different p-vals together
def reFABScalc(pvals):
    r = robjects.r
    r['source']("SMLmetapADJ.R")
    return r("reFABSp(c(" + str(pvals)[1:-1] + "))")   

In [22]:
# Compares original analysis with simulated analysis for nSim
def simulation(GOlist, geneL, nSim, nSites, origAnalysis, method, SMLcutoff, entrezDomains): # GOList is the new go list that we analyze through since we don't want whole GO list, geneL is windowDF
    sitesL = nRandSitesSim(nSites, nSim)
    outputAnalysis = origAnalysis # We will keep updating this dictionary and return when all sims are done 
    
    for sim in range(nSim):
        mapped = geneReadSites(sitesL[sim], geneL, method)
        counters = SMLcounterFAST(GOlist, mapped, SMLcutoff, entrezDomains)
        outputAnalysis = compareGOAnalysis(outputAnalysis, counters)
        
    return sorted(convertAnalysistoFormat(outputAnalysis, nSim), key = lambda x: x[1][1])

In [23]:
# First run that sets up for the simulation
def firstRun(userSitesL, window, method='TSS', simN=100): # userSites must be ordered by chromosome number (List of lists) # Default simN = 10000
    if method == 'TSS':
        geneL = addWindowTSS(window)
        windowDF = addWindowTSS20(window)
    elif method == 'BODY': 
        geneL = addWindowBODY(window)
        windowDF = addWindowBODY20(window)
    else:
        return 'Only TSS or BODY method allowed!'
    
    geneClassification = findSMLdomain(windowDF)
    
    mappedGenes = geneReadSites(userSitesL, geneL, method)
    
    numMapped = len(mappedGenes)
    numsites = 0
    for l in userSitesL:
        numsites += len(l)
    
    entrezDomains = getEntrezDomain(windowDF)
#     return mappedGenes, entrezDomains
    goAnalysis = conductAnalysisFIRST(mappedGenes, geneClassification, entrezDomains)
    newGOlist = getGOfromAnalysis(goAnalysis)
    
    # estimate num sites to sample: numSitesSamp
    nBSL = nRandSitesSim(numsites,5)
    nPrime2 = 0 # Total num genes mapped to figure out nPrime value eventually for estimation
    for BSL in nBSL:
        nPrime2 += len(geneReadSites(BSL, geneL, method))
    nPrime = int(nPrime2 / 5)
    numSitesSamp = int((numMapped * numsites) / nPrime) # Num mapped genes * num sites inputted divided by nPrime 
    
    return simulation(newGOlist, geneL, simN, numSitesSamp, goAnalysis, method, geneClassification, entrezDomains) # GeneL is list of genes with window bounds 

In [24]:
def inANOTB(a,b):
    a_set = set(a) 
    b_set = set(b) 
    return len(list(set(a_set) - set(b_set)))
    

In [25]:
# First run that sets up for the simulation
def adaptiveEnrichmentAnalysis(userSitesL, window, method='TSS'): # userSites must be ordered by chromosome number (List of lists) # Default simN = 10000
    if method == 'TSS':
        geneL = addWindowTSS(window)
        windowDF = addWindowTSS20(window)
    elif method == 'BODY': 
        geneL = addWindowBODY(window)
        windowDF = addWindowBODY20(window)
    else:
        return 'Only TSS or BODY method allowed!'
    
    geneClassification = findSMLdomain(windowDF)
    smallbound = geneClassification[0]
    largebound = geneClassification[1]
    
    mappedGenes = geneReadSites(userSitesL, geneL, method)
    entrezDomains = getEntrezDomain(windowDF)
    smallGenesMAPPED = []# in mapped
    mediumGenesMAPPED = []
    largeGenesMAPPED = []
    for entrez in mappedGenes:
        geneLEN = entrezDomains[entrez]
        if geneLEN < smallbound:
            smallGenesMAPPED.append(entrez)
        elif geneLEN > largebound:
            largeGenesMAPPED.append(entrez)
        else:
            mediumGenesMAPPED.append(entrez)
    
    smallGenesAll = []# in All
    mediumGenesAll = []
    largeGenesAll = []
    
    everygene = []
    for l in windowDF:
        everygene += list(l.entrezid.values)
    for entrez in everygene:
        geneLEN = entrezDomains[entrez]
        if geneLEN < smallbound:
            smallGenesAll.append(entrez)
        elif geneLEN > largebound:
            largeGenesAll.append(entrez)
        else:
            mediumGenesAll.append(entrez)
            
    goAssocGenes = getOntologyID(mappedGenes)
    go2pvals = []
    
    for go in goAssocGenes:
        associated2GO = go2gene[go]
        smallASSOCgo = common_member(smallGenesAll, associated2GO)# in mapped
        mediumASSOCgo = common_member(mediumGenesAll, associated2GO)
        largeASSOCgo = common_member(largeGenesAll, associated2GO)
        
        if len(smallASSOCgo) > 0:
            A = len(common_member(smallASSOCgo, smallGenesMAPPED))
            B = len(smallASSOCgo) - A
            C = inANOTB(smallGenesMAPPED, smallASSOCgo)
            D = len(smallGenesAll) - A - B - C
            smallODDpval = stats.fisher_exact([[A, B], [C, D]])
        else:
            smallODDpval = ('NA','NA')
        
        if len(mediumASSOCgo) > 0:
            E = len(common_member(mediumASSOCgo, mediumGenesMAPPED))
            F = len(mediumASSOCgo) - E
            G = inANOTB(mediumGenesMAPPED, mediumASSOCgo)
            H = len(mediumGenesAll) - E - F - G
            mediumODDpval = stats.fisher_exact([[E, F], [G, H]])
        else:
            mediumODDpval = ('NA','NA')
        
        if len(largeASSOCgo) > 0: 
            I = len(common_member(largeASSOCgo, largeGenesMAPPED))
            J = len(largeASSOCgo) - I
            K = inANOTB(largeGenesMAPPED, largeASSOCgo)
            L = len(largeGenesAll) - I - J - K 
            largeODDpval = stats.fisher_exact([[I, J], [K, L]])
        else:
            largeODDpval = ('NA','NA')

        go2pvals.append([go,smallODDpval, mediumODDpval, largeODDpval])

            
    return go2pvals
    

In [26]:
def adjP(dataSET):
    GOdataSet = dataSET
    goGrp = [[],[],[],[],[],[],[],[],[],[]]
    numAssGrp = [[1], [2], [3], [4], [5, 6], [7, 8], [9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], [34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 182, 183, 185, 187, 188, 189, 190, 191, 193, 194, 196, 198, 200, 201, 202, 203, 204, 205, 206, 208, 210, 211, 212, 213, 214, 216, 217, 218, 219, 221, 225, 226, 229, 232, 234, 235, 236, 238, 239, 240, 242, 244, 249, 250, 251, 252, 254, 262, 263, 264, 265, 268, 272, 273, 278, 280, 282, 285, 290, 295, 297, 301, 307, 308, 310, 311, 312, 313, 315, 324, 329, 330, 334, 343, 344, 347, 354, 360, 362, 364, 366, 368, 371, 374, 375, 385, 387, 395, 397, 405, 409, 411, 413, 420, 424, 427, 431, 439, 441, 457, 458, 467, 474, 481, 483, 491, 496, 502, 504, 505, 524, 529, 537, 540, 547, 548, 552, 565, 578, 605, 610, 611, 667, 670, 689, 703, 706, 812, 816, 859, 927, 960, 973, 980, 1059, 1132, 1134, 1150, 1227, 1304, 1381, 1388, 1456, 1473, 1546, 1778, 1911, 1987, 2163, 2292, 3168, 3637, 4438, 4482, 5026, 5627, 9691]]
    numGO = [5184, 2895, 1743, 1241, 1551, 1018, 1219, 1028, 1020, 1275] # Number of GO terms in each group above.
    
    for go in GOdataSet:
        numAss = len(go2gene[go[0]])
        for i in range(len(numAssGrp)):
            if numAss in numAssGrp[i]:
                goGrp[i].append(go)
    
    pos = 0
    for GOdata in goGrp:
        fishersP = []
#         refabsP = []
        for l in GOdata:
            fishersP.append(l[1][1])
#             refabsP.append(l[1][2])
         
        if len(fishersP) < numGO[pos]:
            numAppend = numGO[pos] - len(fishersP)
            fishersP += [1] * numAppend
#             refabsP += [1] * numAppend
        pos += 1         
        
        reject, fishersPadj, alphacSidak, alphacBonf = padjust(fishersP, method='fdr_bh', is_sorted=False)
#         reject2, refabsPadj, alphacSidak2, alphacBonf2 = padjust(refabsP, method='fdr_bh', is_sorted=False)
        correctedfishers = []
#         correctedrefabs = []
        for i in range(len(GOdata)):
            correctedfishers.append(float(fishersPadj[i]))
#             cohttp://localhost:8890/notebooks/GOfunctionalAnalysis.ipynb#rrectedrefabs.append(float(refabsPadj[i]))
        
        for i in range(len(GOdata)):
            GOdata[i][1].append(correctedfishers[i])
#             GOdata[i][1].append(correctedrefabs[i])
    groupedGO = []
    for i in goGrp:
        for j in i:
            groupedGO.append(j)
    groupedGO = sorted(groupedGO, key = lambda x: x[1][1])
    refabsP = []
    for go in groupedGO:
        refabsP.append(go[1][2])
    reject2, refabsPadj, alphacSidak2, alphacBonf2 = padjust(refabsP, method='fdr_bh', is_sorted=False)
    
    for i in range(len(groupedGO)):
        groupedGO[i][1].append(refabsPadj[i])
    
    return groupedGO # go: [odds, fishersP, refabsP, correctedfishers, correctedrefabs]

In [27]:
# geneAnnotationDF = pd.read_csv('entrez_id/geneAnnotationsDF_Selected_entrezID.csv', sep=',', comment='#', low_memory=False, header=0, names=geneColID)
linSites = pd.read_excel('bindingSites2Test.xlsx', header=0)

In [28]:
chrName = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y']
linSitesCHR = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
i = 0
for cro in chrName:
    newnew = linSites.loc[linSites['chr'] == cro]
    linSitesCHR[i] += list(newnew.site.values)
    i += 1 

In [29]:
outout = adaptiveEnrichmentAnalysis(linSitesCHR, 100000, method='TSS')

In [77]:
randsites = nRandSites([20000, random.randint(100000)])

In [78]:
randomMAPPED = adaptiveEnrichmentAnalysis(randsites, 100000, method='TSS')

In [460]:
allmapped=geneReadSites(randsites, addWindowBODY(10000), method='BODY')

In [461]:
winds=addWindowBODY20(10000)

In [462]:
mapped = []
i = 0 
for ch in winds:
    sites = randsites[i]
    for site in sites:
        mapped += list(ch.loc[(ch['lowB'] < site) & (ch['upperB'] > site)].entrezid.values)
    i += 1 

In [463]:
len(mapped)

9891

In [464]:
len(allmapped)

6038

In [467]:
for i in mapped:
    if i not in allmapped:
        print(i)

100526740
9296


In [80]:
allsites = nRandSitesSim(100000, 1)

In [81]:
mapping = []
for sites in allsites:
    mapped = geneReadSites(sites, addWindowBODY(100000), method='BODY')
    alldomains = getEntrezDomain(addWindowBODY20(100000))
    domains = []
    for entrez in mapped:
        domains.append([entrez,alldomains[entrez]])
    mapping.append(domains)

In [82]:
newmapping = []
for l in mapping:
    l = sorted(l, key=lambda x : x[1])
    newmapping.append(l)

In [83]:
for l in newmapping:
    for gene in l:
        chromo = list(geneAnnotationDF.loc[geneAnnotationDF['entrezid'] == gene[0]].chromosome.values)
        gene += chromo

In [84]:
newmapping[0]

[[414245, 1312, '10', '10'],
 [4536, 1341, 'MT'],
 [4538, 1480, 'MT'],
 [8636, 2395, '9'],
 [100188893, 2502, '6'],
 [116238, 2574, '17'],
 [149473, 2750, '1'],
 [101928527, 2841, '15'],
 [654483, 3091, '16'],
 [5216, 3139, '17'],
 [8358, 3167, '6'],
 [268, 3168, '19'],
 [168620, 3271, '7'],
 [10430, 3406, '19'],
 [6610, 3431, '6'],
 [401251, 3447, '6'],
 [337975, 3486, '21'],
 [51255, 3512, '2'],
 [23625, 3566, '11'],
 [124944, 3689, '17'],
 [51738, 3711, '3'],
 [4192, 3812, '11'],
 [6187, 3828, '16'],
 [100507564, 3963, '1'],
 [116540, 3970, '2'],
 [8969, 4043, '6'],
 [56731, 4103, '20'],
 [6618, 4161, '19'],
 [337967, 4162, '21'],
 [84080, 4179, '16'],
 [57402, 4186, '1'],
 [101929054, 4188, '3'],
 [3199, 4188, '7'],
 [7389, 4249, '1'],
 [27243, 4274, '19'],
 [116173, 4276, '14'],
 [645967, 4298, '6'],
 [554235, 4315, '19'],
 [100529209, 4321, '17'],
 [7311, 4327, '19'],
 [728229, 4395, '22'],
 [100506504, 4432, '1'],
 [337873, 4452, '1'],
 [5720, 4460, '14'],
 [100873952, 4484, '1'

In [60]:
newmapping[4]

[[92806, 3635, '16'],
 [10280, 6557, '9'],
 [10980, 6713, '7'],
 [29925, 8704, '3'],
 [101928603, 9875, '6'],
 [608, 13063, '16'],
 [101927143, 15352, '1'],
 [6517, 15429, '17'],
 [930, 15431, '16'],
 [100507567, 15960, '19'],
 [9532, 17096, '6'],
 [55997, 17364, '2'],
 [728228, 17881, '20'],
 [7384, 18733, '3'],
 [389812, 19717, '9'],
 [64963, 20170, '15'],
 [646625, 21538, '13'],
 [60493, 21577, '20'],
 [55192, 24902, '15'],
 [28954, 25105, '20'],
 [23532, 25367, '22'],
 [54538, 26266, '11'],
 [64091, 26784, '3'],
 [122622, 26852, '14'],
 [391196, 27067, '1'],
 [9092, 27360, '11'],
 [9968, 28310, 'X'],
 [29114, 28352, '3'],
 [100287520, 28502, '11'],
 [102724360, 29007, '19'],
 [79869, 29284, '11'],
 [100507436, 29522, '6'],
 [79879, 29622, '22'],
 [100126572, 30212, '6'],
 [148229, 32230, '19'],
 [54793, 32486, '8'],
 [2978, 33216, '6'],
 [51384, 33572, '7'],
 [2213, 33619, '1'],
 [728957, 33743, '8'],
 [389075, 34833, '2'],
 [126308, 35410, '19'],
 [84693, 35532, '2'],
 [55664, 365

In [69]:
alldomainsx = getEntrezDomain(addWindowBODY20(100000))

In [71]:
newdomains = []
for k in alldomainsx.keys():
    newdomains.append([k,alldomainsx[k]])

In [72]:
newdomains= sorted(newdomains, key=lambda x: x[1])

In [78]:
for l in newdomains:
    chromo = list(geneAnnotationDF.loc[geneAnnotationDF['entrezid'] == l[0]].chromosome.values)
    l += chromo

In [35]:
xxx = adaptiveEnrichmentAnalysis(linSitesCHR, 100000, method='TSS')

In [67]:
goDATA

{'GO:0000001': GOTerm('GO:0000001'):
   id:GO:0000001
   item_id:GO:0000001
   name:mitochondrion inheritance
   namespace:biological_process
   _parents: 2 items
     GO:0048311
     GO:0048308
   parents: 2 items
     GO:0048308	level-04	depth-04	organelle inheritance [biological_process]
     GO:0048311	level-05	depth-05	mitochondrion distribution [biological_process]
   children: 0 items
   level:5
   depth:6
   is_obsolete:False
   alt_ids: 0 items
   relationship: 0 items
   relationship_rev: 0 items
   reldepth:6, 'GO:0000002': GOTerm('GO:0000002'):
   id:GO:0000002
   item_id:GO:0000002
   name:mitochondrial genome maintenance
   namespace:biological_process
   _parents: 1 items
     GO:0007005
   parents: 1 items
     GO:0007005	level-04	depth-04	mitochondrion organization [biological_process]
   children: 1 items
     GO:0033955	level-06	depth-06	mitochondrial DNA inheritance [biological_process]
   level:5
   depth:5
   is_obsolete:False
   alt_ids: 0 items
   relationship: 

In [68]:
listsofGO = ['GO:2000377',
'GO:0030544',
'GO:0070198',
'GO:0006620',
'GO:0035249',
'GO:0070412',
'GO:0045662',
'GO:0036414',
'GO:0008126',
'GO:0009925',
'GO:0042289',
'GO:0042472',
'GO:0006891',
'GO:0048015',
'GO:0032496',
'GO:0015893',
'GO:0016805',
'GO:0045444',
'GO:0035335',
'GO:0006355',
'GO:0000002',
'GO:0006284',
'GO:0097322',
'GO:0051216',
'GO:0019953',
'GO:0002064',
'GO:0030903',
'GO:0048714',
'GO:0030317',
'GO:0061106',
'GO:0001975',
'GO:0004993',
'GO:1902305',
'GO:0019532',
'GO:0035094',
'GO:0032922',
'GO:0030322',
'GO:0070016',
'GO:0035195',
'GO:0014894',
'GO:0006364',
'GO:0042974',
'GO:0004938',
'GO:0007281',
'GO:0045499',
'GO:0050750',
'GO:2000491',
'GO:0032466',
'GO:0035098',
'GO:0016538',
'GO:0005540',
'GO:0043552',
'GO:0016192',
'GO:0046789',
'GO:0051087',
'GO:0000722',
'GO:0034354',
'GO:0004935',
'GO:0033745',
'GO:0008028',
'GO:0016446',
'GO:0002576',
'GO:0034605',
'GO:0004016',
'GO:0070097',
'GO:0008207',
'GO:2001135',
'GO:0042593',
'GO:0042834',
'GO:0000334',
'GO:0005634',
'GO:0030971',
'GO:0030091',
'GO:0070847',
'GO:0090084',
'GO:0001093',
'GO:0007217',
'GO:0032725',
'GO:1990454',
'GO:2000269',
'GO:2000737',
'GO:0040010',
'GO:0008517',
'GO:0042149',
'GO:0009303',
'GO:0005328',
'GO:0005623',
'GO:0033993',
'GO:0072385',
'GO:0002003',
'GO:0002544',
'GO:0071300',
'GO:0035330',
'GO:0004190',
'GO:0071549',
'GO:0032580',
'GO:0019531',
'GO:2000503',
'GO:0006582',
'GO:2000352',
'GO:0051793',
'GO:0071222',
'GO:0043422',
'GO:0071820',
'GO:0031122',
'GO:0035136',
'GO:0005020',
'GO:0043537',
'GO:0071676',
'GO:0038113',
'GO:0030488',
'GO:0006970',
'GO:0045907',
'GO:0016922',
'GO:0060996',
'GO:0042267',
'GO:0008092',
'GO:0007050',
'GO:0060339',
'GO:0015081',
'GO:0003256',
'GO:0006896',
'GO:0034620',
'GO:0042119',
'GO:0097536',
'GO:0036438',
'GO:0010718',
'GO:0006413',
'GO:1990869',
'GO:0060213',
'GO:0000784',
'GO:0045598',
'GO:0050692',
'GO:0034371',
'GO:0045579',
'GO:0071480',
'GO:2000272',
'GO:0000228',
'GO:0016671',
'GO:0035722',
'GO:0022900',
'GO:0050911',
'GO:0018024',
'GO:0060037',
'GO:0071353',
'GO:0030674',
'GO:0071459',
'GO:1902476',
'GO:0046688',
'GO:0045672',
'GO:0048012',
'GO:0055026',
'GO:0097352',
'GO:0030424',
'GO:0032471',
'GO:0042908',
'GO:0051894',
'GO:1902691',
'GO:0031994',
'GO:0018242',
'GO:0005604',
'GO:0097150',
'GO:0006044',
'GO:0043473',
'GO:0004519',
'GO:0034115',
'GO:0014059',
'GO:0055078',
'GO:0000980',
'GO:2001237',
'GO:2000812',
'GO:0017156',
'GO:0051575',
'GO:1905521',
'GO:1904338',
'GO:0120020',
'GO:0060136',
'GO:0042631',
'GO:1902257',
'GO:0006812',
'GO:1990089',
'GO:0010494',
'GO:0009880',
'GO:0030521',
'GO:0009584',
'GO:0031618',
'GO:0034332',
'GO:0051897',
'GO:0010614',
'GO:0036302',
'GO:0006531',
'GO:0043652',
'GO:0060216',
'GO:0042761',
'GO:0016324',
'GO:0004559',
'GO:0021903',
'GO:0097062',
'GO:0043687',
'GO:0009100',
'GO:0005543',
'GO:0007018',
'GO:0003084',
'GO:0017124',
'GO:2000525',
'GO:0048016',
'GO:0006090',
'GO:0004551',
'GO:0060323',
'GO:1905069',
'GO:0042698',
'GO:0007601',
'GO:0006886',
'GO:0042010',
'GO:0043330',
'GO:0097197',
'GO:2000669',
'GO:0019956',
'GO:0014047',
'GO:1904343',
'GO:0097001',
'GO:0002605',
'GO:0031594',
'GO:0034111',
'GO:0006264',
'GO:0006839',
'GO:0001047',
'GO:0042053',
'GO:0071005',
'GO:0030449',
'GO:0051971',
'GO:0032835',
'GO:0005903',
'GO:0070221',
'GO:0006310',
'GO:0019228',
'GO:0019079',
'GO:0070534',
'GO:0097225',
'GO:0008536',
'GO:0000164',
'GO:0016197',
'GO:0001709',
'GO:0060071',
'GO:0021615',
'GO:0042073',
'GO:0051898',
'GO:0005811',
'GO:0018101',
'GO:0008083',
'GO:0051973',
'GO:1903803',
'GO:0060686',
'GO:0050678',
'GO:0034452',
'GO:0034101',
'GO:0007140',
'GO:0001765',
'GO:0007621',
'GO:0019372',
'GO:0045335',
'GO:0031092',
'GO:0045199',
'GO:0071379',
'GO:0045190',
'GO:0043063',
'GO:0003062',
'GO:0005871',
'GO:0006289',
'GO:0006497',
'GO:0032812',
'GO:0090657',
'GO:0005576',
'GO:0005184',
'GO:0002286',
'GO:1903896',
'GO:0038096',
'GO:1901990',
'GO:0036503',
'GO:0009755',
'GO:0031100',
'GO:0003205',
'GO:0031966',
'GO:0071332',
'GO:0035176',
'GO:0034721',
'GO:0010820',
'GO:2000251',
'GO:0051908',
'GO:0001574',
'GO:0071404',
'GO:0055114',
'GO:0001887',
'GO:0048240',
'GO:1903286',
'GO:0060999',
'GO:0042475',
'GO:0061418',
'GO:0032229',
'GO:0042538',
'GO:0019216',
'GO:0008009',
'GO:0005654',
'GO:0038095',
'GO:0060438',
'GO:0018027',
'GO:0061009',
'GO:0045617',
'GO:0030512',
'GO:0034617',
'GO:0071398',
'GO:0030522',
'GO:0043276',
'GO:0036449',
'GO:0021558',
'GO:0030291',
'GO:0002121',
'GO:0060218',
'GO:0050681',
'GO:0042100',
'GO:0030178',
'GO:0035878',
'GO:0002755',
'GO:2000391',
'GO:0006509',
'GO:0031307',
'GO:0061614',
'GO:0000077',
'GO:0035815',
'GO:0048936',
'GO:0006559',
'GO:2000107',
'GO:0016363',
'GO:0030162',
'GO:0043097',
'GO:0018009',
'GO:0051151',
'GO:0043531',
'GO:0070262',
'GO:0019825',
'GO:0042158',
'GO:0043219',
'GO:0032154',
'GO:0034702',
'GO:0007029',
'GO:0060174',
'GO:0070544',
'GO:0008121',
'GO:0030284',
'GO:0060333',
'GO:0043268',
'GO:1905653',
'GO:0003725',
'GO:0014068',
'GO:0051482',
'GO:0030111',
'GO:0004528',
'GO:0043200',
'GO:0032355',
'GO:1905552',
'GO:0051707',
'GO:0032794',
'GO:0046786',
'GO:0032926',
'GO:0014050',
'GO:0000977',
'GO:0031016',
'GO:0062023',
'GO:0043564',
'GO:0071400',
'GO:0061436',
'GO:1903489',
'GO:0004602',
'GO:0016358',
'GO:0031492',
'GO:0050877',
'GO:0060416',
'GO:0034237',
'GO:0035666',
'GO:0002070',
'GO:2000510',
'GO:0031588',
'GO:0001228',
'GO:0031902',
'GO:0019787',
'GO:1900273',
'GO:0042760',
'GO:0007257',
'GO:0045454',
'GO:0051089',
'GO:0007224',
'GO:0006508',
'GO:0061656',
'GO:0045109',
'GO:0022405',
'GO:0006513',
'GO:0090150',
'GO:0017134',
'GO:0032386',
'GO:0039702',
'GO:0003007',
'GO:0036500',
'GO:1903039',
'GO:0030275',
'GO:0019085',
'GO:0031432',
'GO:0071482',
'GO:0021520',
'GO:0035627',
'GO:0006955',
'GO:0010698',
'GO:0051602',
'GO:0061026',
'GO:0044388',
'GO:0003197',
'GO:0006584',
'GO:0016055',
'GO:0043551',
'GO:0021670',
'GO:0003756',
'GO:0031045',
'GO:0085020',
'GO:0070224',
'GO:0005834',
'GO:0034338',
'GO:0070358',
'GO:0055092',
'GO:0002068',
'GO:0034121',
'GO:0060591',
'GO:0031069',
'GO:0044342',
'GO:0015267',
'GO:0019900',
'GO:0021999',
'GO:0030011',
'GO:0035091',
'GO:0035509',
'GO:0099590',
'GO:0031529',
'GO:0048732',
'GO:2000795',
'GO:0003417',
'GO:0007204',
'GO:0032762',
'GO:0035994',
'GO:0035312',
'GO:0030136',
'GO:0003810',
'GO:0070525',
'GO:0006897',
'GO:0005242',
'GO:0090382',
'GO:0031581',
'GO:0032543',
'GO:0090673',
'GO:0008403',
'GO:0034113',
'GO:0030513',
'GO:0035701',
'GO:1902236',
'GO:0008525',
'GO:2000786',
'GO:0048738',
'GO:0018149',
'GO:0016248',
'GO:0006665',
'GO:0034383',
'GO:0008652',
'GO:0016328',
'GO:0040029',
'GO:0002029',
'GO:0031698',
'GO:0060575',
'GO:0045176',
'GO:0007342',
'GO:1902895',
'GO:0030177',
'GO:0072520',
'GO:0070423',
'GO:0090148',
'GO:0031394',
'GO:0035910',
'GO:0010951',
'GO:0045061',
'GO:0006520',
'GO:0004473',
'GO:0019722',
'GO:0003735',
'GO:0045860',
'GO:0071310',
'GO:0009952',
'GO:0042220',
'GO:0033162',
'GO:0043006',
'GO:0140009',
'GO:0030593',
'GO:0032148',
'GO:0032655',
'GO:0016308',
'GO:0006995',
'GO:0019976',
'GO:0033206',
'GO:0005938',
'GO:0042130',
'GO:0014905',
'GO:0060170',
'GO:0009083',
'GO:2000667',
'GO:1904220',
'GO:0051864',
'GO:0031209',
'GO:0030539',
'GO:0061003',
'GO:0051142',
'GO:1990166',
'GO:1990450',
'GO:0009117',
'GO:0045060',
'GO:0032755',
'GO:0001654',
'GO:1905580',
'GO:0009449',
'GO:0043507',
'GO:0002526',
'GO:0043124',
'GO:0097021',
'GO:0048105',
'GO:0035814',
'GO:0034382',
'GO:0010314',
'GO:0035710',
'GO:0004652',
'GO:1904016',
'GO:0050766',
'GO:0036312',
'GO:1901228',
'GO:0008203',
'GO:0019034',
'GO:0050728',
'GO:0002926',
'GO:0015248',
'GO:0098826',
'GO:0048477',
'GO:0043372',
'GO:0046464',
'GO:0009374',
'GO:1904885',
'GO:0047496',
'GO:0098794',
'GO:0010754',
'GO:0090734',
'GO:0016079',
'GO:0050771',
'GO:0050772',
'GO:0042795',
'GO:2001238',
'GO:0002429',
'GO:0004521',
'GO:0021570',
'GO:2000781',
'GO:0031901',
'GO:0050673',
'GO:0000226',
'GO:0032426',
'GO:0048487',
'GO:0050862',
'GO:0033063',
'GO:0005159',
'GO:0030506',
'GO:0060267',
'GO:0061608',
'GO:0008266',
'GO:0004984',
'GO:0035198',
'GO:0045296',
'GO:1902945',
'GO:0036459',
'GO:1902042',
'GO:0031226',
'GO:0019748',
'GO:0032348',
'GO:2000252',
'GO:0034184',
'GO:0060045',
'GO:0048715',
'GO:0005254',
'GO:0061014',
'GO:0048286',
'GO:0032480',
'GO:0031702',
'GO:0003289',
'GO:0021784',
'GO:0004470',
'GO:0043539',
'GO:0033229',
'GO:1990381',
'GO:0004511',
'GO:0015986',
'GO:0031398',
'GO:0005275',
'GO:0000408',
'GO:0031267',
'GO:0043029',
'GO:0060212',
'GO:0033280',
'GO:0050767',
'GO:0071205',
'GO:0043198',
'GO:0046671',
'GO:0008020',
'GO:0021828',
'GO:0051056',
'GO:0050890',
'GO:0042271',
'GO:0003334',
'GO:0070207',
'GO:1902842',
'GO:0005102',
'GO:0005759',
'GO:0043392',
'GO:0007492',
'GO:0014912',
'GO:0010452',
'GO:0006635',
'GO:0033077',
'GO:0051645',
'GO:0051290',
'GO:0051220',
'GO:0001664',
'GO:0032515',
'GO:0048103',
'GO:0008745',
'GO:0048608',
'GO:0046329',
'GO:0008109',
'GO:0030509',
'GO:0007221',
'GO:0048669',
'GO:0090102',
'GO:0030833',
'GO:0007616',
'GO:0045586',
'GO:0045839',
'GO:0003676',
'GO:0000932',
'GO:0090559',
'GO:0097191',
'GO:0021540',
'GO:0031410',
'GO:0035097',
'GO:0016514',
'GO:0099185',
'GO:0048596',
'GO:0032596',
'GO:0016126',
'GO:0061462',
'GO:0034205',
'GO:0050830',
'GO:0008380',
'GO:1990535',
'GO:0005783',
'GO:1902475',
'GO:0008542',
'GO:1904385',
'GO:0045621',
'GO:0016491',
'GO:0003351',
'GO:0070126',
'GO:0071409',
'GO:0016344',
'GO:2000480',
'GO:0006853',
'GO:0001934',
'GO:0010226',
'GO:0071375',
'GO:0008437',
'GO:0034260',
'GO:0051798',
'GO:0034497',
'GO:0002088',
'GO:0050999',
'GO:0097067',
'GO:1903044',
'GO:0045588',
'GO:0003014',
'GO:0046037',
'GO:0005977',
'GO:0097060',
'GO:0001942',
'GO:0070300',
'GO:0030864',
'GO:0071782',
'GO:0015186',
'GO:0007189',
'GO:0001161',
'GO:0001650',
'GO:0046874',
'GO:0009642',
'GO:0051647',
'GO:0045954',
'GO:0061037',
'GO:0032091',
'GO:0060666',
'GO:0030071',
'GO:0045145',
'GO:2000394',
'GO:0051177',
'GO:1905004',
'GO:0071461',
'GO:0030224',
'GO:0008047',
'GO:0045198',
'GO:0070577',
'GO:0010667',
'GO:0072197',
'GO:0048484',
'GO:0004888',
'GO:1904430',
'GO:0004672',
'GO:0007566',
'GO:0045880',
'GO:0001558',
'GO:0051891',
'GO:0042984',
'GO:0097250',
'GO:0009913',
'GO:0032328',
'GO:0019901',
'GO:0099509',
'GO:0035331',
'GO:0004411',
'GO:0019367',
'GO:0030666',
'GO:0042594',
'GO:0004911',
'GO:0006029',
'GO:0005789',
'GO:0001779',
'GO:0033743',
'GO:0009635',
'GO:0060687',
'GO:0010557',
'GO:0007617',
'GO:1904378',
'GO:0060100',
'GO:1904929',
'GO:1905012',
'GO:0035054',
'GO:0004653',
'GO:0015630',
'GO:0005839',
'GO:0048251',
'GO:0072610',
'GO:2000522',
'GO:0004879',
'GO:0060731',
'GO:0010744',
'GO:0005283',
'GO:0000186',
'GO:0060488',
'GO:0005788',
'GO:0007032',
'GO:0048205',
'GO:0048678',
'GO:0046710',
'GO:0003323',
'GO:0007628',
'GO:0030235',
'GO:0098815',
'GO:0044849',
'GO:0001822',
'GO:0015293',
'GO:0003707',
'GO:0005615',
'GO:0002430',
'GO:0061049',
'GO:0000290',
'GO:0030314',
'GO:0006986',
'GO:0019789',
'GO:0048407',
'GO:0003093',
'GO:0048711',
'GO:2000310',
'GO:0090316',
'GO:0016874',
'GO:0071910',
'GO:0050796',
'GO:0032956',
'GO:0021549',
'GO:0034450',
'GO:0043353',
'GO:0097602',
'GO:0030141',
'GO:0043295',
'GO:1903712',
'GO:0005262',
'GO:0048027',
'GO:0042391',
'GO:0008188',
'GO:0060528',
'GO:0006657',
'GO:0032808',
'GO:2000379',
'GO:0030170',
'GO:0006606',
'GO:0002690',
'GO:2001027',
'GO:0003300',
'GO:0009967',
'GO:0071391',
'GO:0005537',
'GO:0040018',
'GO:0009931',
'GO:0070821',
'GO:0006974',
'GO:0042627',
'GO:0043433',
'GO:0050996',
'GO:0042369',
'GO:0006006',
'GO:0004129',
'GO:0006464',
'GO:0031965',
'GO:1903671',
'GO:0001158',
'GO:0005784',
'GO:0035050',
'GO:0047631',
'GO:0048754',
'GO:0043621',
'GO:0042910',
'GO:0090051',
'GO:0098793',
'GO:0001957',
'GO:0043536',
'GO:0097209',
'GO:0007338',
'GO:0051996',
'GO:0055012',
'GO:0071805',
'GO:0000422',
'GO:0008305',
'GO:0031089',
'GO:0007223',
'GO:0051117',
'GO:0010260',
'GO:0042531',
'GO:1904815',
'GO:0043516',
'GO:0097535',
'GO:0071837',
'GO:0061086',
'GO:0009253',
'GO:0004867',
'GO:0015272',
'GO:0033691',
'GO:0004571',
'GO:0042790',
'GO:0002922',
'GO:0032914',
'GO:0009409',
'GO:0001947',
'GO:0038111',
'GO:1905274',
'GO:0099160',
'GO:0035115',
'GO:0032786',
'GO:0035909',
'GO:0008430',
'GO:0035357',
'GO:0006687',
'GO:0033600',
'GO:0044458',
'GO:0022857',
'GO:0048635',
'GO:0043569',
'GO:0007584',
'GO:0001666',
'GO:0070106',
'GO:0098974',
'GO:1900272',
'GO:0030183',
'GO:0030670',
'GO:0048017',
'GO:0048488',
'GO:0018105',
'GO:1903672',
'GO:0098609',
'GO:0006836',
'GO:0015879',
'GO:0004995',
'GO:0071902',
'GO:0005088',
'GO:0003180',
'GO:0070971',
'GO:0030014',
'GO:0008306',
'GO:0072488',
'GO:0004518',
'GO:0035457',
'GO:0000724',
'GO:0007569',
'GO:0045668',
'GO:0098641',
'GO:0007141',
'GO:0045338',
'GO:0017015',
'GO:0033198',
'GO:0045630',
'GO:1903507',
'GO:0035023',
'GO:0000309',
'GO:0000723',
'GO:1990234',
'GO:0000166',
'GO:0005524',
'GO:0070317',
'GO:0010224',
'GO:0016925',
'GO:0090398',
'GO:0000777',
'GO:1903351',
'GO:1904886',
'GO:0010043',
'GO:0060750',
'GO:0019827',
'GO:0071260',
'GO:0048384',
'GO:0061337',
'GO:0030866',
'GO:0038024',
'GO:0008201',
'GO:0048872',
'GO:0038178',
'GO:1903135',
'GO:0006821',
'GO:0004471',
'GO:0034976',
'GO:0048662',
'GO:0050727',
'GO:0032811',
'GO:0034199',
'GO:0002042',
'GO:0045028',
'GO:0070053',
'GO:0045415',
'GO:0007030',
'GO:0042788',
'GO:0021571',
'GO:0006412',
'GO:0034440',
'GO:0021502',
'GO:0050679',
'GO:0006895',
'GO:0032045',
'GO:0045725',
'GO:0014042',
'GO:0042423',
'GO:0005001',
'GO:0006935',
'GO:0003684',
'GO:0050851',
'GO:0015180',
'GO:1902514',
'GO:0043654',
'GO:0030169',
'GO:0000795',
'GO:0046658',
'GO:0007596',
'GO:0005635',
'GO:2000065',
'GO:0030010',
'GO:0015914',
'GO:0008209',
'GO:0042276',
'GO:0004359',
'GO:0002551',
'GO:0000793',
'GO:0043967',
'GO:0030669',
'GO:0072602',
'GO:0030238',
'GO:0060314',
'GO:1902715',
'GO:0070419',
'GO:0010243',
'GO:0009566',
'GO:0006469',
'GO:0061178',
'GO:0060426',
'GO:0035338',
'GO:0048514',
'GO:0070895',
'GO:0030247',
'GO:0036186',
'GO:0004535',
'GO:0050706',
'GO:0060691',
'GO:0032900',
'GO:0090383',
'GO:0044300',
'GO:0003143',
'GO:0043171',
'GO:0099173',
'GO:0048617',
'GO:0032006',
'GO:0034638',
'GO:0005856',
'GO:0016485',
'GO:0006811',
'GO:0010390',
'GO:0060052',
'GO:0007157',
'GO:0000187',
'GO:0016575',
'GO:0050689',
'GO:0001221',
'GO:0072178',
'GO:0097153',
'GO:0043567',
'GO:0001188',
'GO:0001818',
'GO:0099601',
'GO:0045821',
'GO:0019058',
'GO:0099060',
'GO:0031861',
'GO:0038115',
'GO:0048864',
'GO:0001602',
'GO:0051539',
'GO:0046324',
'GO:0072133',
'GO:0043406',
'GO:0006801',
'GO:0048743',
'GO:0019510',
'GO:0003887',
'GO:0048170',
'GO:0016235',
'GO:1904724',
'GO:0051276',
'GO:0033169',
'GO:0030295',
'GO:0033300',
'GO:0042995',
'GO:0046676',
'GO:0015031',
'GO:0061001',
'GO:1901622',
'GO:1903753',
'GO:0031981',
'GO:0006598',
'GO:0097213',
'GO:0008239',
'GO:0010216',
'GO:0071363',
'GO:0004515',
'GO:0060662',
'GO:0021984',
'GO:0034993',
'GO:0006572',
'GO:1900246',
'GO:0009395',
'GO:0001817',
'GO:0120147',
'GO:0009966',
'GO:0010642',
'GO:0060429',
'GO:0015698',
'GO:0002141',
'GO:0061073',
'GO:0090074',
'GO:0042470',
'GO:0006357',
'GO:0046040',
'GO:0034975',
'GO:0071934',
'GO:0030215',
'GO:0055117',
'GO:0031228',
'GO:0016323',
'GO:0035860',
'GO:0044325',
'GO:0003290',
'GO:0009235',
'GO:0005815',
'GO:0001952',
'GO:0046703',
'GO:0004478',
'GO:1903955',
'GO:0034622',
'GO:0005905',
'GO:0097151',
'GO:0055024',
'GO:0071565',
'GO:0003158',
'GO:0021707',
'GO:0034045',
'GO:0060088',
'GO:0051259',
'GO:1900453',
'GO:0050995',
'GO:0003229',
'GO:0042421',
'GO:0006012',
'GO:1900165',
'GO:0005198',
'GO:0004447',
'GO:0034056',
'GO:0030041',
'GO:0008297',
'GO:0042802',
'GO:0045602',
'GO:0043303',
'GO:1990966',
'GO:1905289',
'GO:0015459',
'GO:0007099',
'GO:0003215',
'GO:1902513',
'GO:0015918',
'GO:1905671',
'GO:1990696',
'GO:0010585',
'GO:0010633',
'GO:0005509',
'GO:0010827',
'GO:1905515',
'GO:0099523',
'GO:0010389',
'GO:0001701',
'GO:0097498',
'GO:0046689',
'GO:0001825',
'GO:1901534',
'GO:0051250',
'GO:0045795',
'GO:0070052',
'GO:0005536',
'GO:0008179',
'GO:0090026',
'GO:0051571',
'GO:0005876',
'GO:0005671',
'GO:0005353',
'GO:0010212',
'GO:0007521',
'GO:0006405',
'GO:0045822',
'GO:0006642',
'GO:0099147',
'GO:0042048',
'GO:0008015',
'GO:0004252',
'GO:0008511',
'GO:0034446',
'GO:0006338',
'GO:0043066',
'GO:0016234',
'GO:0006515',
'GO:0031225',
'GO:0000788',
'GO:0007264',
'GO:0035524',
'GO:0046469',
'GO:0008199',
'GO:0031683',
'GO:0030851',
'GO:0042981',
'GO:0046983',
'GO:0048786',
'GO:2001295',
'GO:2000179',
'GO:0090656',
'GO:0030334',
'GO:0021612',
'GO:0010803',
'GO:0001504',
'GO:0001938',
'GO:0006688',
'GO:0060489',
'GO:0051930',
'GO:0006585',
'GO:0033588',
'GO:0007015',
'GO:0008320',
'GO:0071673',
'GO:0016604',
'GO:0070473',
'GO:0042800',
'GO:0030947',
'GO:0006281',
'GO:0032869',
'GO:0008233',
'GO:1990357',
'GO:0018423',
'GO:0051497',
'GO:0032848',
'GO:0006555',
'GO:0060548',
'GO:0005496',
'GO:2000664',
'GO:0048566',
'GO:0051292',
'GO:0007517',
'GO:0009743',
'GO:0035987',
'GO:0046596',
'GO:0000421',
'GO:0050773',
'GO:0045010',
'GO:0051260',
'GO:0090314',
'GO:0043195',
'GO:0010838',
'GO:0008081',
'GO:0048148',
'GO:0010800',
'GO:0016272',
'GO:0018026',
'GO:0007611',
'GO:0019104',
'GO:0048843',
'GO:0045656',
'GO:0003700',
'GO:0051025',
'GO:0005882',
'GO:0033146',
'GO:0001540',
'GO:0006361',
'GO:0032986',
'GO:0051289',
'GO:0007009',
'GO:0005923',
'GO:0090543',
'GO:2000482',
'GO:0000132',
'GO:0045746',
'GO:0071907',
'GO:0031983',
'GO:0043169',
'GO:0006479',
'GO:0005525',
'GO:0045059',
'GO:0014701',
'GO:0090575',
'GO:0034395',
'GO:0006334',
'GO:0050829',
'GO:0051536',
'GO:0016597',
'GO:0035591',
'GO:0034504',
'GO:0051970',
'GO:1901386',
'GO:0038117',
'GO:0010996',
'GO:0045153',
'GO:0001939',
'GO:0031300',
'GO:0072282',
'GO:0005892',
'GO:0072107',
'GO:0030431',
'GO:0060259',
'GO:0070915',
'GO:0003281',
'GO:0071108',
'GO:0000175',
'GO:0022417',
'GO:0060290',
'GO:0044257',
'GO:0046629',
'GO:0015195',
'GO:0022400',
'GO:0010906',
'GO:0015651',
'GO:0046328',
'GO:0005484',
'GO:1903543',
'GO:0030206',
'GO:0072010',
'GO:0001974',
'GO:0071474',
'GO:0007276',
'GO:0005768',
'GO:0043370',
'GO:0017022',
'GO:0032244',
'GO:0045348',
'GO:0030554',
'GO:0005747',
'GO:0016791',
'GO:0007187',
'GO:0031668',
'GO:0033089',
'GO:0009651',
'GO:0035299',
'GO:0003785',
'GO:0006367',
'GO:0051321',
'GO:0001755',
'GO:1901838',
'GO:0005000',
'GO:0007284',
'GO:0097192',
'GO:0033630',
'GO:0035791',
'GO:0051899',
'GO:0071437',
'GO:0008253',
'GO:0022617',
'GO:0005351',
'GO:0031894',
'GO:0051425',
'GO:1901796',
'GO:0044212',
'GO:0003730',
'GO:0004364',
'GO:0043619',
'GO:2000734',
'GO:0035748',
'GO:0048266',
'GO:0001739',
'GO:0042167',
'GO:0032420',
'GO:0043542',
'GO:0005979',
'GO:0051983',
'GO:0060749',
'GO:0042323',
'GO:0006107',
'GO:0075341',
'GO:0048660',
'GO:0005874',
'GO:0005246',
'GO:0002548',
'GO:0042755',
'GO:0001662',
'GO:0000151',
'GO:0034116',
'GO:0006122',
'GO:0006873',
'GO:0060560',
'GO:0045777',
'GO:0016595',
'GO:0061564',
'GO:0002572',
'GO:0051592',
'GO:0051446',
'GO:0006691',
'GO:0036313',
'GO:1903760',
'GO:0042169',
'GO:0003724',
'GO:0009615',
'GO:0038109',
'GO:0050953',
'GO:0051209',
'GO:0044062',
'GO:0003105',
'GO:0070966',
'GO:0031460',
'GO:0051082',
'GO:0032092',
'GO:0001702',
'GO:0006766',
'GO:0003952',
'GO:0006501',
'GO:0004609',
'GO:0005802',
'GO:0042826',
'GO:0009653',
'GO:0019371',
'GO:0005506',
'GO:0045540',
'GO:0008286',
'GO:0043236',
'GO:0070530',
'GO:0032720',
'GO:1904251',
'GO:0035757',
'GO:0000293',
'GO:0031532',
'GO:0000045',
'GO:0051256',
'GO:0035898',
'GO:0016019',
'GO:0004415',
'GO:0005743',
'GO:0042462',
'GO:0005596',
'GO:0008630',
'GO:0046875',
'GO:0032924',
'GO:0060326',
'GO:0071535',
'GO:0010460',
'GO:0006621',
'GO:0048485',
'GO:0007626',
'GO:0005080',
'GO:0070410',
'GO:0005111',
'GO:0032958',
'GO:0005518',
'GO:0008328',
'GO:0008584',
'GO:0048844',
'GO:2000703',
'GO:1905168',
'GO:0006366',
'GO:0045027',
'GO:0010975',
'GO:0102338',
'GO:0004814',
'GO:0008373',
'GO:0006396',
'GO:0010172',
'GO:0050769',
'GO:0004075',
'GO:0006027',
'GO:0003096',
'GO:0030617',
'GO:0001609',
'GO:0070628',
'GO:0097100',
'GO:0001768',
'GO:0010596',
'GO:0034626',
'GO:0050690',
'GO:2001234',
'GO:0035690',
'GO:0060158',
'GO:0090161',
'GO:0038121',
'GO:0050919',
'GO:0048639',
'GO:0097029',
'GO:0033590',
'GO:0006874',
'GO:0022612',
'GO:0015701',
'GO:0043666',
'GO:1905555',
'GO:0015671',
'GO:0007017',
'GO:0050840',
'GO:0060534',
'GO:0007155',
'GO:0007084',
'GO:0048665',
'GO:0060287',
'GO:0042326',
'GO:0007173',
'GO:0070301',
'GO:0008023',
'GO:0016505',
'GO:0045294',
'GO:0043034',
'GO:0009266',
'GO:1903358',
'GO:0051402',
'GO:0044666',
'GO:0070700',
'GO:0036164',
'GO:0008073',
'GO:0008645',
'GO:0043560',
'GO:0035735',
'GO:1902773',
'GO:0010863',
'GO:0060740',
'GO:2001301',
'GO:0015030',
'GO:0007165',
'GO:0046425',
'GO:0000502',
'GO:0021575',
'GO:0042982',
'GO:0008378',
'GO:0036477',
'GO:0070411',
'GO:0098839',
'GO:0097110',
'GO:0002175',
'GO:0045471',
'GO:0043534',
'GO:0070382',
'GO:0000118',
'GO:0005746',
'GO:0004553',
'GO:0019433',
'GO:0048156',
'GO:0071347',
'GO:0030033',
'GO:0004013',
'GO:0047860',
'GO:0045988',
'GO:0085029',
'GO:0051653',
'GO:0043394',
'GO:0006914',
'GO:0034684',
'GO:0030016',
'GO:0099508',
'GO:0008385',
'GO:0004089',
'GO:0008544',
'GO:0010606',
'GO:1900026',
'GO:0001931',
'GO:0045333',
'GO:0035729',
'GO:0032391',
'GO:0042500',
'GO:0002353',
'GO:1902937',
'GO:0007389',
'GO:0043312',
'GO:0016407',
'GO:0005355',
'GO:0038093',
'GO:0048880',
'GO:2000060',
'GO:0060402',
'GO:0032649',
'GO:0055075',
'GO:0045428',
'GO:0070062',
'GO:0016309',
'GO:0032147',
'GO:0006420',
'GO:0019902',
'GO:0097016',
'GO:0007093',
'GO:0034589',
'GO:0005154',
'GO:0032781',
'GO:0061141',
'GO:0046427',
'GO:0090090',
'GO:0000209',
'GO:0043586',
'GO:0043280',
'GO:0071377',
'GO:0022409',
'GO:0019048',
'GO:0006084',
'GO:0042180',
'GO:0051271',
'GO:0051146',
'GO:0106016',
'GO:0008948',
'GO:0008821',
'GO:0032332',
'GO:0015837',
'GO:0005044',
'GO:0006646',
'GO:0016020',
'GO:0003690',
'GO:0004527',
'GO:1900153',
'GO:0035196',
'GO:0008180',
'GO:0000027',
'GO:0070166',
'GO:0030342',
'GO:0016567',
'GO:0007010',
'GO:0014003',
'GO:0043966',
'GO:0033574',
'GO:0070006',
'GO:0031528',
'GO:0032827',
'GO:0002087',
'GO:0019852',
'GO:0004222',
'GO:0030173',
'GO:0030307',
'GO:2001235',
'GO:0005134',
'GO:0008206',
'GO:0006511',
'GO:0002244',
'GO:0001607',
'GO:0097326',
'GO:0004683',
'GO:0060279',
'GO:0071596',
'GO:0072659',
'GO:0010575',
'GO:0031076',
'GO:0043587',
'GO:1903546',
'GO:0000159',
'GO:0003085',
'GO:0010977',
'GO:0098978',
'GO:0031526',
'GO:0030330',
'GO:0033017',
'GO:0008376',
'GO:1990075',
'GO:0090331',
'GO:0008022',
'GO:0033693',
'GO:0042405',
'GO:2000650',
'GO:0051895',
'GO:0044305',
'GO:0060397',
'GO:0020037',
'GO:0070059',
'GO:0032981',
'GO:0042803',
'GO:0032991',
'GO:0000775',
'GO:0044772',
'GO:0031419',
'GO:0072593',
'GO:0005769',
'GO:0043434',
'GO:0002262',
'GO:0006597',
'GO:0051371',
'GO:0071949',
'GO:0004074',
'GO:0060672',
'GO:0007212',
'GO:0033762',
'GO:0003208',
'GO:1905643',
'GO:0009435',
'GO:0032469',
'GO:0035579',
'GO:0046914',
'GO:0045124',
'GO:0007625',
'GO:0097026',
'GO:0001658',
'GO:0019005',
'GO:0042501',
'GO:0030856',
'GO:1903076',
'GO:0070373',
'GO:0070851',
'GO:0010595',
'GO:0005720',
'GO:0023014',
'GO:0060337',
'GO:0016887',
'GO:0070634',
'GO:0071276',
'GO:0019730',
'GO:0046966',
'GO:0022848',
'GO:0032266',
'GO:0071456',
'GO:0051024',
'GO:0102756',
'GO:0016208',
'GO:0036064',
'GO:0001757',
'GO:0055010',
'GO:0042136',
'GO:0001656',
'GO:0031145',
'GO:0043086',
'GO:0070371',
'GO:0098943',
'GO:0045055',
'GO:0060392',
'GO:0038023',
'GO:0030948',
'GO:1903375',
'GO:0060349',
'GO:0015278',
'GO:0070474',
'GO:0032809',
'GO:0071442',
'GO:0035257',
'GO:0016477',
'GO:0003824',
'GO:0005668',
'GO:0004532',
'GO:0098901',
'GO:0009267',
'GO:0000139',
'GO:0072134',
'GO:0036159',
'GO:0051781',
'GO:0001516',
'GO:0072676',
'GO:0006629',
'GO:0031652',
'GO:0035240',
'GO:0052895',
'GO:0035116',
'GO:0060090',
'GO:0030020',
'GO:0050853',
'GO:0043504',
'GO:0042254',
'GO:0005813',
'GO:1901166',
'GO:0010972',
'GO:0009411',
'GO:0000052',
'GO:0006091',
'GO:0015867',
'GO:1905065',
'GO:0035329',
'GO:0047391',
'GO:0002674',
'GO:0006661',
'GO:0051149',
'GO:0009060',
'GO:0040014',
'GO:0035356',
'GO:0043378',
'GO:0043523',
'GO:0005125',
'GO:1905166',
'GO:0031755',
'GO:0044294',
'GO:0046975',
'GO:0005694',
'GO:0046466',
'GO:0048706',
'GO:0003148',
'GO:0010941',
'GO:0033365',
'GO:0071320',
'GO:0042733',
'GO:0003274',
'GO:0032967',
'GO:2000373',
'GO:0045765',
'GO:0042418',
'GO:0051015',
'GO:0042552',
'GO:0009888',
'GO:0072709',
'GO:0070201',
'GO:0001227',
'GO:0005009',
'GO:1902202',
'GO:0031579',
'GO:0015116',
'GO:0030182',
'GO:0007565',
'GO:0008343',
'GO:2000342',
'GO:0042713',
'GO:0042711',
'GO:0046850',
'GO:0008354',
'GO:0007052',
'GO:0097194',
'GO:0019992',
'GO:0070936',
'GO:0033270',
'GO:0051382',
'GO:1904154',
'GO:0034755',
'GO:0043113',
'GO:0060770',
'GO:0009311',
'GO:0032873',
'GO:0009298',
'GO:0008499',
'GO:0003989',
'GO:0007520',
'GO:0048208',
'GO:0072012',
'GO:0060490',
'GO:0016042',
'GO:0034063',
'GO:2000347',
'GO:0016780',
'GO:0007169',
'GO:0060384',
'GO:0048178',
'GO:0032703',
'GO:0014065',
'GO:0097443',
'GO:0098962',
'GO:0051127',
'GO:0033157',
'GO:0033619',
'GO:0034629',
'GO:0050428',
'GO:0045184',
'GO:0034968',
'GO:0006898',
'GO:0001823',
'GO:0003847',
'GO:0031298',
'GO:0001547',
'GO:0006123',
'GO:0051443',
'GO:0071407',
'GO:0042883',
'GO:0001816',
'GO:0045652',
'GO:0042641',
'GO:1905312',
'GO:1990573',
'GO:0045879',
'GO:0016482',
'GO:0035914',
'GO:0035265',
'GO:0071242',
'GO:1901223',
'GO:0044232',
'GO:0090251',
'GO:0006972',
'GO:0010385',
'GO:0007423',
'GO:0031047',
'GO:0015825',
'GO:0004464',
'GO:0007266',
'GO:0042509',
'GO:0032715',
'GO:0060340',
'GO:0009247',
'GO:1901491',
'GO:0045048',
'GO:0045732',
'GO:2000611',
'GO:0032474',
'GO:1904659',
'GO:0031623',
'GO:0001843',
'GO:0015175',
'GO:0031334',
'GO:0004714',
'GO:0045202',
'GO:0006869',
'GO:0010269',
'GO:0005547',
'GO:1900740',
'GO:0005762',
'GO:0001913',
'GO:1903799',
'GO:0097324',
'GO:0008270',
'GO:0001886',
'GO:0032870',
'GO:0014902',
'GO:0097491',
'GO:0030672',
'GO:0060973',
'GO:0060412',
'GO:0098712',
'GO:0010001',
'GO:0036297',
'GO:0021754',
'GO:0051572',
'GO:0039529',
'GO:0007160',
'GO:0006302',
'GO:0060487',
'GO:0050849',
'GO:2000114',
'GO:0007612',
'GO:0050431',
'GO:0006699',
'GO:0034360',
'GO:0045666',
'GO:1903936',
'GO:0007188',
'GO:0048730',
'GO:0014911',
'GO:0007249',
'GO:0030032',
'GO:0050770',
'GO:0034243',
'GO:0050510',
'GO:0071731',
'GO:0030018',
'GO:0046985',
'GO:0030850',
'GO:0050900',
'GO:1990227',
'GO:0030296',
'GO:0051000',
'GO:0019217',
'GO:0032911',
'GO:2001243',
'GO:0051379',
'GO:0003382',
'GO:0005347',
'GO:0060317',
'GO:0000447',
'GO:0046677',
'GO:0030154',
'GO:0001657',
'GO:0050512',
'GO:0010862',
'GO:0030031',
'GO:0002949',
'GO:0051938',
'GO:0003774',
'GO:0043420',
'GO:0014066',
'GO:0060715',
'GO:1904268',
'GO:0006663',
'GO:0097546',
'GO:1990718',
'GO:0050852',
'GO:0030673',
'GO:0008603',
'GO:1903762',
'GO:0005201',
'GO:0008290',
'GO:0034138',
'GO:0043129',
'GO:0001740',
'GO:0060390',
'GO:0004142',
'GO:0000398',
'GO:0031641',
'GO:1902287',
'GO:0034614',
'GO:2001256',
'GO:0032849',
'GO:0099505',
'GO:0008139',
'GO:0043622',
'GO:0071140',
'GO:2001204',
'GO:0042127',
'GO:0070860',
'GO:0030659',
'GO:0071636',
'GO:0031624',
'GO:0030131',
'GO:0070932',
'GO:0003203',
'GO:0002756',
'GO:0021542',
'GO:1905564',
'GO:0019904',
'GO:0070779',
'GO:0016311',
'GO:0051881',
'GO:0001576',
'GO:0070888',
'GO:1904998',
'GO:0001837',
'GO:0050821',
'GO:0021793',
'GO:0034644',
'GO:0007632',
'GO:0046208',
'GO:0042809',
'GO:0000780',
'GO:0032963',
'GO:0021675',
'GO:0098696',
'GO:0099184',
'GO:0051393',
'GO:0032700',
'GO:0031424',
'GO:0042035',
'GO:0043491',
'GO:1900182',
'GO:0005893',
'GO:0000375',
'GO:0071277',
'GO:0006614',
'GO:0030165',
'GO:0031493',
'GO:0014032',
'GO:0042297',
'GO:0055129',
'GO:0010705',
'GO:0001935',
'GO:0030176',
'GO:0048013',
'GO:0071818',
'GO:1990418',
'GO:0034363',
'GO:0007500',
'GO:0030100',
'GO:0043525',
'GO:0010485',
'GO:0031305',
'GO:0034191',
'GO:0005801',
'GO:0046902',
'GO:0010033',
'GO:0008282',
'GO:0071345',
'GO:0005925',
'GO:2000291',
'GO:0030030',
'GO:0030326',
'GO:0043025',
'GO:0006397',
'GO:0010628',
'GO:0009450',
'GO:0002224',
'GO:1903588',
'GO:2000479',
'GO:0006977',
'GO:0032436',
'GO:0016327',
'GO:0098989',
'GO:1903077',
'GO:0045931',
'GO:0060374',
'GO:0007268',
'GO:0030964',
'GO:0032201',
'GO:0098686',
'GO:0030027',
'GO:0043547',
'GO:0048568',
'GO:0015813',
'GO:0007530',
'GO:0004679',
'GO:0009414',
'GO:0022625',
'GO:0042629',
'GO:0005886',
'GO:0070374',
'GO:0060334',
'GO:0000987',
'GO:0045618',
'GO:0009087',
'GO:0006655',
'GO:0051091',
'GO:0000278',
'GO:0005875',
'GO:0106015',
'GO:0031018',
'GO:0045815',
'GO:0036510',
'GO:0005884',
'GO:0015811',
'GO:0042277',
'GO:0035470',
'GO:0016010',
'GO:0016278',
'GO:0097490',
'GO:0019087',
'GO:0030216',
'GO:0001502',
'GO:0019897',
'GO:0070507',
'GO:0003682',
'GO:0034362',
'GO:0046621',
'GO:1903898',
'GO:2000601',
'GO:0032049',
'GO:0031012',
'GO:0072182',
'GO:0006487',
'GO:0010046',
'GO:0035864',
'GO:0004803',
'GO:0040037',
'GO:0006342',
'GO:0048873',
'GO:1904798',
'GO:2001141',
'GO:0043209',
'GO:0008307',
'GO:0021960',
'GO:0003179',
'GO:0071230',
'GO:0048870',
'GO:0071392',
'GO:0035727',
'GO:0006477',
'GO:0010332',
'GO:0030336',
'GO:0003777',
'GO:0005911',
'GO:0090023',
'GO:0010506',
'GO:0031871',
'GO:0040008',
'GO:0034441',
'GO:0048069',
'GO:0001894',
'GO:0042246',
'GO:0002931',
'GO:0060038',
'GO:0048753',
'GO:0032934',
'GO:0032509',
'GO:0051782',
'GO:0001010',
'GO:0003254',
'GO:0015190',
'GO:0030901',
'GO:0048666',
'GO:0071318',
'GO:0000776',
'GO:0032270',
'GO:0070613',
'GO:0060676',
'GO:0048565',
'GO:0048745',
'GO:2001271',
'GO:0051092',
'GO:0033605',
'GO:2000778',
'GO:0050731',
'GO:0006605',
'GO:0046930',
'GO:1904184',
'GO:0017016',
'GO:0050776',
'GO:0051403',
'GO:0004615',
'GO:0008528',
'GO:0097190',
'GO:0046330',
'GO:0044798',
'GO:0006644',
'GO:0045608',
'GO:0090527',
'GO:0051865',
'GO:0048185',
'GO:0006521',
'GO:0060316',
'GO:0070273',
'GO:0016338',
'GO:0034551',
'GO:0010757',
'GO:0003429',
'GO:1904288',
'GO:0034198',
'GO:0018158',
'GO:0051301',
'GO:0002009',
'GO:0045064',
'GO:0008146',
'GO:0031274',
'GO:0045724',
'GO:0048306',
'GO:0007422',
'GO:0050220',
'GO:0016045',
'GO:1900100',
'GO:0090305',
'GO:0030321',
'GO:0030902',
'GO:0016601',
'GO:0048469',
'GO:0034097',
'GO:0009898',
'GO:0001921',
'GO:0098869',
'GO:0048599',
'GO:0016573',
'GO:2000547',
'GO:0032620',
'GO:0010621',
'GO:0047298',
'GO:0021637',
'GO:0061825',
'GO:0004176',
'GO:0004758',
'GO:0007200',
'GO:0006730',
'GO:0034616',
'GO:0004385',
'GO:0018153',
'GO:0120043',
'GO:0005324',
'GO:0090179',
'GO:0006526',
'GO:0001889',
'GO:0072740',
'GO:0008594',
'GO:0060716',
'GO:0001973',
'GO:0001756',
'GO:0071499',
'GO:0001836',
'GO:0042609',
'GO:0001867',
'GO:0046320',
'GO:0032587',
'GO:0005796',
'GO:1903621',
'GO:0001527',
'GO:0005912',
'GO:0000902',
'GO:0001992',
'GO:0017025',
'GO:0019985',
'GO:0046339',
'GO:0005899',
'GO:0035148',
'GO:2000300',
'GO:0015643',
'GO:0010259',
'GO:0042776',
'GO:2001224',
'GO:0006108',
'GO:0016705',
'GO:1901687',
'GO:0032868',
'GO:0001835',
'GO:1901342',
'GO:0008757',
'GO:0001954',
'GO:0010467',
'GO:1903599',
'GO:0036126',
'GO:0072643',
'GO:0005885',
'GO:1901536',
'GO:0005765',
'GO:0034713',
'GO:0002260',
'GO:0006473',
'GO:0016310',
'GO:0017119',
'GO:0042592',
'GO:0006265',
'GO:1904951',
'GO:0018108',
'GO:0052314',
'GO:0000915',
'GO:0006884',
'GO:0044323',
'GO:0043588',
'GO:1902166',
'GO:0005794',
'GO:0003713',
'GO:0097487',
'GO:0097756',
'GO:0001968',
'GO:0032041',
'GO:0070837',
'GO:0006808',
'GO:0009791',
'GO:1903755',
'GO:0017148',
'GO:0033227',
'GO:0051270',
'GO:0034599',
'GO:0006953',
'GO:1903078',
'GO:0038001',
'GO:0015718',
'GO:0030175',
'GO:2000121',
'GO:1990738',
'GO:0008017',
'GO:0055037',
'GO:0060041',
'GO:0070987',
'GO:0045964',
'GO:0102336',
'GO:0019773',
'GO:0017046',
'GO:0030955',
'GO:1902232',
'GO:0006570',
'GO:0006417',
'GO:2000227',
'GO:0001678',
'GO:0070886',
'GO:0014075',
'GO:0000786',
'GO:0005776',
'GO:1903202',
'GO:0036464',
'GO:0033292',
'GO:0030888',
'GO:0043375',
'GO:0051639',
'GO:0070402',
'GO:0045211',
'GO:0070662',
'GO:0008219',
'GO:0033130',
'GO:0050291',
'GO:0030511',
'GO:0010719',
'GO:0051085',
'GO:0046777',
'GO:0007195',
'GO:0002793',
'GO:0032008',
'GO:0070208',
'GO:0004930',
'GO:0050482',
'GO:0016056',
'GO:0060463',
'GO:0021681',
'GO:0005637',
'GO:0072112',
'GO:0008272',
'GO:0003924',
'GO:0045793',
'GO:0007031',
'GO:0001760',
'GO:0090263',
'GO:0003344',
'GO:0003108',
'GO:0046592',
'GO:0031666',
'GO:0006954',
'GO:0046513',
'GO:0072592',
'GO:0031998',
'GO:1902186',
'GO:0006486',
'GO:0030855',
'GO:0016300',
'GO:0090260',
'GO:0061085',
'GO:0044322',
'GO:0003727',
'GO:0008625',
'GO:0000407',
'GO:0018107',
'GO:0001791',
'GO:0044295',
'GO:0005881',
'GO:0051018',
'GO:0031210',
'GO:0003272',
'GO:0046717',
'GO:0034361',
'GO:0033878',
'GO:2000146',
'GO:0043149',
'GO:0038116',
'GO:0071599',
'GO:0005913',
'GO:0035589',
'GO:0003723',
'GO:0034505',
'GO:1900034',
'GO:0120072',
'GO:0002407',
'GO:2000311',
'GO:2000050',
'GO:0006631',
'GO:0006024',
'GO:0070561',
'GO:0007411',
'GO:0098797',
'GO:0044877',
'GO:0099061',
'GO:0004862',
'GO:0030433',
'GO:0045892',
'GO:2000327',
'GO:0032482',
'GO:0070257',
'GO:0031227',
'GO:0022898',
'GO:0016279',
'GO:0043087',
'GO:0032402',
'GO:2001065',
'GO:0001519',
'GO:0015378',
'GO:0001893',
'GO:0032144',
'GO:2000035',
'GO:0004622',
'GO:0006909',
'GO:0001750',
'GO:0001967',
'GO:0062000',
'GO:0060738',
'GO:0043401',
'GO:0007214',
'GO:0008409',
'GO:0014823',
'GO:0046854',
'GO:0014069',
'GO:0006814',
'GO:0032416',
'GO:0002457',
'GO:0042987',
'GO:0038083',
'GO:0043407',
'GO:0034379',
'GO:0060509',
'GO:0051247',
'GO:0061144',
'GO:0000922',
'GO:0045121',
'GO:0016592',
'GO:1905475',
'GO:0070064',
'GO:0006990',
'GO:0000307',
'GO:0004185',
'GO:0035725',
'GO:0051480',
'GO:0046627',
'GO:0008360',
'GO:0071333',
'GO:0006595',
'GO:0005883',
'GO:0038102',
'GO:2001022',
'GO:1902093',
'GO:0031643',
'GO:0006612',
'GO:0050873',
'GO:0051599',
'GO:0060993',
'GO:0098038',
'GO:1902603',
'GO:2000049',
'GO:0007519',
'GO:0006890',
'GO:0008235',
'GO:0003677',
'GO:0003917',
'GO:0050896',
'GO:0060706',
'GO:0032281',
'GO:0048814',
'GO:0048863',
'GO:1902268',
'GO:0051574',
'GO:0097718',
'GO:1990830',
'GO:0042159',
'GO:0030279',
'GO:0000790',
'GO:0005741',
'GO:0042304',
'GO:0006094',
'GO:0004298',
'GO:0010447',
'GO:0051090',
'GO:0120162',
'GO:0099056',
'GO:0008289',
'GO:0048511',
'GO:1903045',
'GO:0070848',
'GO:0097038',
'GO:0061817',
'GO:0004309',
'GO:0006798',
'GO:0030479',
'GO:1990918',
'GO:0030132',
'GO:0017137',
'GO:0051923',
'GO:0009306',
'GO:0071774',
'GO:0035137',
'GO:0030056',
'GO:0030534',
'GO:0034314',
'GO:0016051',
'GO:0046332',
'GO:0021987',
'GO:1902253',
'GO:0060486',
'GO:0071682',
'GO:0031941',
'GO:1904036',
'GO:1990904',
'GO:0036295',
'GO:0000275',
'GO:0048038',
'GO:0015194',
'GO:0007193',
'GO:0071493',
'GO:0045648',
'GO:0015184',
'GO:0051902',
'GO:0005319',
'GO:0061549',
'GO:0047238',
'GO:0033268',
'GO:1905934',
'GO:0035900',
'GO:0015889',
'GO:0006904',
'GO:0048679',
'GO:0043398',
'GO:0002455',
'GO:0043679',
'GO:0005178',
'GO:0006805',
'GO:0048762',
'GO:0086004',
'GO:0015108',
'GO:0000792',
'GO:0045859',
'GO:0045600',
'GO:0043651',
'GO:0001726',
'GO:0034146',
'GO:0043647',
'GO:0060743',
'GO:0005131',
'GO:0035307',
'GO:0016266',
'GO:0000993',
'GO:0010613',
'GO:0030134',
'GO:0035036',
'GO:0035469',
'GO:0030194',
'GO:0004310',
'GO:0051650',
'GO:0090266',
'GO:0038191',
'GO:0006270',
'GO:0002027',
'GO:0035333',
'GO:0060010',
'GO:0031017',
'GO:0071385',
'GO:0072136',
'GO:0043408',
'GO:0044267',
'GO:0070098',
'GO:0003779',
'GO:0002021',
'GO:0001501',
'GO:0005761',
'GO:0045292',
'GO:1903553',
'GO:0050693',
'GO:0000400',
'GO:0007163',
'GO:0043697',
'GO:0060271',
'GO:0030148',
'GO:0036134',
'GO:0043497',
'GO:0001102',
'GO:0017112',
'GO:0030913',
'GO:0030518',
'GO:0032754',
'GO:0042461',
'GO:0106091',
'GO:1902857',
'GO:0061673',
'GO:0046597',
'GO:0051412',
'GO:0030953',
'GO:0052548',
'GO:0005739',
'GO:0016746',
'GO:0033601',
'GO:0030496',
'GO:2000811',
'GO:1904996',
'GO:0005313',
'GO:0005515',
'GO:0005798',
'GO:1903670',
'GO:0000053',
'GO:0007202',
'GO:0048546',
'GO:0098554',
'GO:0008361',
'GO:0006590',
'GO:0015234',
'GO:0000165',
'GO:0016202',
'GO:0006303',
'GO:0031730',
'GO:1990907',
'GO:0050808',
'GO:0034625',
'GO:0061832',
'GO:0031253',
'GO:0015379',
'GO:0007614',
'GO:0050807',
'GO:0001917',
'GO:0071349',
'GO:0004722',
'GO:0000096',
'GO:0071295',
'GO:0031752',
'GO:2000147',
'GO:0005149',
'GO:0022407',
'GO:0007296',
'GO:0032735',
'GO:0030335',
'GO:0009408',
'GO:0033962',
'GO:0071901',
'GO:0021537',
'GO:0051287',
'GO:0060253',
'GO:0060173',
'GO:2000317',
'GO:0097750',
'GO:0030218',
'GO:0015171',
'GO:0008301',
'GO:0050847',
'GO:0031647',
'GO:0035519',
'GO:0001895',
'GO:0009165',
'GO:0030968',
'GO:0010763',
'GO:0097711',
'GO:0030282',
'GO:0030214',
'GO:0071475',
'GO:0038114',
'GO:0060017',
'GO:0000079',
'GO:0042148',
'GO:1903818',
'GO:0002326',
'GO:0046982',
'GO:0002408',
'GO:0006749',
'GO:0045596',
'GO:0051569',
'GO:0051968',
'GO:0071786',
'GO:0045779',
'GO:0017108',
'GO:0060539',
'GO:0072125',
'GO:2000249',
'GO:0071458',
'GO:0046881',
'GO:0000981',
'GO:1901216',
'GO:0046718',
'GO:0021772',
'GO:0044304',
'GO:0061384',
'GO:0006915',
'GO:0043085',
'GO:0007275',
'GO:0071560',
'GO:0030163',
'GO:0060338',
'GO:0007154',
'GO:0060596',
'GO:0097084',
'GO:0003867',
'GO:0034364',
'GO:0007019',
'GO:0005930',
'GO:0007219',
'GO:0001671',
'GO:0048705',
'GO:0097484',
'GO:0001772',
'GO:0032088',
'GO:0004713',
'GO:0000287',
'GO:0008670',
'GO:0097746',
'GO:0006195',
'GO:2000170',
'GO:0021904',
'GO:0003880',
'GO:0007162',
'GO:0061743',
'GO:0060385',
'GO:0061844',
'GO:0034359',
'GO:0007549',
'GO:0001838',
'GO:0099645',
'GO:0060411',
'GO:0019898',
'GO:0016529',
'GO:0030421',
'GO:0043011',
'GO:1904262',
'GO:1901837',
'GO:0042307',
'GO:0048278',
'GO:0035984',
'GO:0010008',
'GO:0048589',
'GO:0004175',
'GO:0050780',
'GO:0046686',
'GO:0033076',
'GO:0048147',
'GO:0033148',
'GO:0008228',
'GO:0048846',
'GO:0005031',
'GO:0030510',
'GO:0045259',
'GO:0006659',
'GO:0000785',
'GO:0034333',
'GO:0033689',
'GO:0009887',
'GO:0030514',
'GO:0007191',
'GO:0032993',
'GO:0000149',
'GO:0006813',
'GO:0047696',
'GO:0070498',
'GO:1903508',
'GO:0070840',
'GO:0006625',
'GO:0002023',
'GO:0031440',
'GO:0016247',
'GO:2001184',
'GO:0045977',
'GO:0071346',
'GO:0016605',
'GO:0097065',
'GO:0009611',
'GO:0015826',
'GO:1904813',
'GO:0060307',
'GO:0000184',
'GO:0043022',
'GO:0014067',
'GO:0006569',
'GO:0032145',
'GO:0042403',
'GO:0006362',
'GO:0071312',
'GO:0016493',
'GO:0000082',
'GO:0005104',
'GO:0042043',
'GO:0046755',
'GO:1902952',
'GO:0034236',
'GO:0010594',
'GO:0009636',
'GO:0021694',
'GO:0050774',
'GO:0004721',
'GO:0090611',
'GO:0042562',
'GO:0120163',
'GO:0001618',
'GO:0046684',
'GO:0098970',
'GO:0032653',
'GO:0006694',
'GO:0045840',
'GO:0010498',
'GO:0060079',
'GO:0055089',
'GO:0055072',
'GO:0015179',
'GO:0031648',
'GO:0007267',
'GO:0075713',
'GO:0030949',
'GO:0019894',
'GO:0070588',
'GO:0045590',
'GO:0097284',
'GO:0042059',
'GO:0097427',
'GO:0043532',
'GO:0015037',
'GO:0003712',
'GO:0005577',
'GO:0045177',
'GO:0060021',
'GO:0045505',
'GO:0007286',
'GO:0005158',
'GO:0070253',
'GO:0036109',
'GO:0044211',
'GO:0034067',
'GO:0004864',
'GO:0010801',
'GO:0060465',
'GO:0052894',
'GO:0033194',
'GO:0055088',
'GO:0071897',
'GO:1901381',
'GO:2000134',
'GO:0030037',
'GO:1902897',
'GO:0052901',
'GO:0005641',
'GO:0006936',
'GO:0002885',
'GO:0019731',
'GO:0015755',
'GO:0070979',
'GO:0044255',
'GO:0043123',
'GO:0016137',
'GO:0016747',
'GO:0061952',
'GO:1904382',
'GO:0042060',
'GO:0002437',
'GO:0097550',
'GO:0014053',
'GO:1990380',
'GO:0048265',
'GO:0045047',
'GO:0070328',
'GO:0070830',
'GO:0090286',
'GO:2001241',
'GO:0007213',
'GO:0002369',
'GO:0099524',
'GO:0034635',
'GO:0006623',
'GO:0071467',
'GO:0051896',
'GO:0045003',
'GO:0005096',
'GO:0006816',
'GO:2001240',
'GO:0038061',
'GO:0072049',
'GO:0021557',
'GO:0010664',
'GO:0008444',
'GO:0043627',
'GO:0005764',
'GO:0060393',
'GO:0005790',
'GO:0061575',
'GO:0035019',
'GO:0010559',
'GO:1902038',
'GO:1902511',
'GO:0006351',
'GO:0019221',
'GO:0007602',
'GO:0003829',
'GO:0002053',
'GO:0002520',
'GO:0035371',
'GO:0007568',
'GO:0007620',
'GO:0031995',
'GO:0060675',
'GO:0030133',
'GO:0001944',
'GO:0002360',
'GO:0002227',
'GO:0005249',
'GO:0005245',
'GO:0021983',
'GO:0097120',
'GO:0000122',
'GO:0030665',
'GO:0045814',
'GO:0097350',
'GO:0071168',
'GO:0060741',
'GO:0060445',
'GO:0001570',
'GO:0031295',
'GO:0008568',
'GO:0010457',
'GO:0060070',
'GO:1903561',
'GO:0019083',
'GO:0014704',
'GO:1901211',
'GO:0016459',
'GO:0017154',
'GO:2000543',
'GO:0008137',
'GO:0051967',
'GO:0061551',
'GO:0030660',
'GO:0061309',
'GO:0002371',
'GO:0036258',
'GO:0031061',
'GO:0031333',
'GO:0061512',
'GO:0051787',
'GO:2000172',
'GO:0005868',
'GO:0007097',
'GO:0001661',
'GO:0033141',
'GO:0098810',
'GO:0036060',
'GO:0005546',
'GO:0008237',
'GO:2000607',
'GO:0097524',
'GO:0042975',
'GO:0045159',
'GO:0071356',
'GO:0061024',
'GO:1903764',
'GO:0061635',
'GO:0015842',
'GO:0032204',
'GO:0060055',
'GO:0051924',
'GO:0090024',
'GO:0031490',
'GO:0086080',
'GO:0061630',
'GO:0007340',
'GO:0042626',
'GO:0008519',
'GO:0016528',
'GO:0010832',
'GO:0044306',
'GO:0031682',
'GO:0060964',
'GO:0003729',
'GO:0000976',
'GO:0019911',
'GO:0008198',
'GO:0005978',
'GO:0004402',
'GO:0042383',
'GO:0045741',
'GO:0050998',
'GO:0071169',
'GO:0044346',
'GO:0070001',
'GO:0097720',
'GO:0006306',
'GO:0001778',
'GO:0005737',
'GO:0010759',
'GO:0005581',
'GO:0060828',
'GO:0010507',
'GO:0043084',
'GO:1901029',
'GO:0042723',
'GO:0030548',
'GO:0048483',
'GO:0002320',
'GO:0045636',
'GO:0007190',
'GO:1904504',
'GO:2000617',
'GO:2000679',
'GO:0097466',
'GO:0045624',
'GO:0005230',
'GO:0003678',
'GO:1904556',
'GO:0065003',
'GO:0072734',
'GO:0060324',
'GO:0033010',
'GO:0010922',
'GO:0010886',
'GO:0009897',
'GO:0060395',
'GO:0034774',
'GO:0006110',
'GO:0005657',
'GO:0032330',
'GO:0055064',
'GO:0070372',
'GO:0046085',
'GO:2000378',
'GO:0045787',
'GO:0043032',
'GO:0060748',
'GO:1903206',
'GO:0007409',
'GO:0019835',
'GO:0004197',
'GO:0071316',
'GO:0030168',
'GO:0007059',
'GO:2000010',
'GO:0005770',
'GO:0002116',
'GO:0030155',
'GO:0072332',
'GO:0009986',
'GO:0048037',
'GO:0044351',
'GO:0030060',
'GO:0010894',
'GO:0033700',
'GO:0046847',
'GO:0015629',
'GO:0006750',
'GO:0004379',
'GO:0051660',
'GO:0005844',
'GO:0045197',
'GO:0099582',
'GO:1904380',
'GO:0031982',
'GO:0003906',
'GO:0033602',
'GO:0003714',
'GO:0030289',
'GO:0043410',
'GO:0022841',
'GO:0042415',
'GO:0034386',
'GO:0009314',
'GO:0003341',
'GO:0004725',
'GO:0019471',
'GO:0046326',
'GO:0032736',
'GO:1902187',
'GO:0006633',
'GO:0016740',
'GO:0007098',
'GO:0042147',
'GO:0042445',
'GO:0017125',
'GO:0046931',
'GO:0051792',
'GO:0098911',
'GO:0034165',
'GO:0007413',
'GO:0012501',
'GO:0046599',
'GO:0021861',
'GO:0017059',
'GO:0000086',
'GO:2001020',
'GO:1904158',
'GO:0009312',
'GO:0000939',
'GO:0030308',
'GO:0031463',
'GO:0050910',
'GO:0035758',
'GO:0035481',
'GO:0005123',
'GO:0005381',
'GO:0034447',
'GO:0016788',
'GO:0042924',
'GO:0036435',
'GO:0036498',
'GO:0070813',
'GO:1904450',
'GO:0045776',
'GO:0006013',
'GO:0035108',
'GO:0021766',
'GO:2001244',
'GO:0016971',
'GO:0030217',
'GO:0031252',
'GO:0031625',
'GO:0000289',
'GO:0055056',
'GO:0048839',
'GO:0032481',
'GO:0004177',
'GO:0002142',
'GO:0045494',
'GO:0046882',
'GO:0051384',
'GO:0008607',
'GO:0009612',
'GO:0043197',
'GO:0150020',
'GO:0005544',
'GO:0050544',
'GO:0000940',
'GO:0034080',
'GO:0060065',
'GO:0004715',
'GO:0045332',
'GO:0032259',
'GO:0060644',
'GO:0051568',
'GO:0030426',
'GO:0048821',
'GO:0097327',
'GO:0042745',
'GO:0010751',
'GO:0002725',
'GO:0005814',
'GO:0097022',
'GO:0001708',
'GO:0048387',
'GO:0034590',
'GO:0005200',
'GO:0007186',
'GO:0021591',
'GO:0031929',
'GO:0009968',
'GO:0015196',
'GO:0006325',
'GO:0080025',
'GO:0061036',
'GO:0006690',
'GO:0030199',
'GO:0015990',
'GO:0001164',
'GO:0000281',
'GO:1904322',
'GO:0019955',
'GO:0043325',
'GO:0030667',
'GO:1902285',
'GO:0030478',
'GO:0033189',
'GO:0038162',
'GO:0008631',
'GO:0001892',
'GO:0001937',
'GO:0050882',
'GO:0035234',
'GO:0035578',
'GO:0019888',
'GO:0030501',
'GO:0038110',
'GO:0005975',
'GO:0033504',
'GO:0005721',
'GO:0002043',
'GO:0070102',
'GO:0045930',
'GO:0042118',
'GO:0052106',
'GO:0043559',
'GO:0008065',
'GO:1901208',
'GO:0008016',
'GO:0006406',
'GO:0042113',
'GO:0003190',
'GO:0032753',
'GO:1903551',
'GO:0043565',
'GO:0090103',
'GO:0001659',
'GO:0071287',
'GO:0000978',
'GO:0030520',
'GO:0042551',
'GO:0005777',
'GO:2000637',
'GO:0001963',
'GO:0050680',
'GO:0097418',
'GO:0006959',
'GO:0034220',
'GO:1900029',
'GO:0061146',
'GO:0002606',
'GO:0036486',
'GO:0006879',
'GO:0009416',
'GO:1990264',
'GO:0031234',
'GO:0046580',
'GO:0003844',
'GO:0002181',
'GO:0014049',
'GO:0035556',
'GO:0009629',
'GO:0051306',
'GO:0051386',
'GO:0060612',
'GO:0042997',
'GO:0021785',
'GO:0097025',
'GO:0042587',
'GO:0042102',
'GO:0030101',
'GO:0004668',
'GO:0048646',
'GO:0097009',
'GO:0007218',
'GO:0015036',
'GO:0034711',
'GO:0018008',
'GO:0005516',
'GO:0016574',
'GO:0016540',
'GO:0071526',
'GO:0030425',
'GO:0005730',
'GO:0005369',
'GO:0021644',
'GO:0043535',
'GO:0021940',
'GO:2000978',
'GO:0061061',
'GO:0044344',
'GO:0009055',
'GO:0097242',
'GO:0010700',
'GO:0010468',
'GO:0048813',
'GO:0045008',
'GO:1904399',
'GO:0030124',
'GO:0048625',
'GO:0051607',
'GO:0003691',
'GO:0048011',
'GO:0032415',
'GO:0031999',
'GO:0048712',
'GO:0007005',
'GO:0034329',
'GO:0008526',
'GO:0010738',
'GO:0015026',
'GO:0090162',
'GO:0031201',
'GO:0044206',
'GO:0006893',
'GO:0033499',
'GO:0061631',
'GO:0001786',
'GO:0007049',
'GO:0006493',
'GO:0014031',
'GO:0102337',
'GO:0005225',
'GO:0032825',
'GO:2000526',
'GO:0010923',
'GO:0048471',
'GO:0042393',
'GO:0060523',
'GO:0035035',
'GO:0030276',
'GO:2000353',
'GO:0045429',
'GO:0044803',
'GO:0007399',
'GO:0030015',
'GO:0001541',
'GO:0071542',
'GO:0043395',
'GO:1903779',
'GO:0048280',
'GO:0042632',
'GO:0061700',
'GO:0061713',
'GO:0000900',
'GO:0050707',
'GO:0033116',
'GO:1900106',
'GO:0032526',
'GO:0045944',
'GO:0032680',
'GO:0004034',
'GO:0012507',
'GO:0005989',
'GO:0051721',
'GO:0048747',
'GO:0005778',
'GO:0022008',
'GO:0060586',
'GO:0070293',
'GO:0060389',
'GO:0001829',
'GO:0031103',
'GO:0033209',
'GO:0032689',
'GO:0060912',
'GO:0051496',
'GO:0001665',
'GO:0019886',
'GO:0045582',
'GO:0001669',
'GO:0016787',
'GO:0005829',
'GO:0070757',
'GO:0006260',
'GO:0071875',
'GO:0007497',
'GO:0007229',
'GO:0004878',
'GO:0031146',
'GO:0032452',
'GO:0033128',
'GO:0006790',
'GO:0021801',
'GO:0061098',
'GO:0048704',
'GO:0046331',
'GO:0090502',
'GO:0048008',
'GO:0071556',
'GO:0097503',
'GO:0042535',
'GO:0031587',
'GO:0060087',
'GO:0006824',
'GO:0031397',
'GO:0042117',
'GO:0019957',
'GO:1904899',
'GO:1900006',
'GO:2000020',
'GO:0006865',
'GO:0030643',
'GO:0002102',
'GO:0060485',
'GO:0032760',
'GO:0030036',
'GO:0006868',
'GO:0005522',
'GO:0055038',
'GO:0018243',
'GO:0043524',
'GO:0048538',
'GO:0060252',
'GO:0070935',
'GO:0140078',
'GO:0035799',
'GO:0034123',
'GO:0004674',
'GO:0017053',
'GO:0042752',
'GO:0044650',
'GO:0044314',
'GO:0006418',
'GO:0032252',
'GO:0051561',
'GO:0000778',
'GO:0005179',
'GO:0004569',
'GO:0035577',
'GO:0002479',
'GO:0002040',
'GO:0003091',
'GO:0045671',
'GO:1901611',
'GO:0098632',
'GO:0060164',
'GO:0047499',
'GO:0048010',
'GO:0045104',
'GO:0016768',
'GO:0035014',
'GO:0097454',
'GO:0047372',
'GO:1903038',
'GO:0044320',
'GO:0090737',
'GO:0001520',
'GO:0030198',
'GO:0030203',
'GO:0032508',
'GO:2000845',
'GO:0007623',
'GO:0061626',
'GO:0097381',
'GO:0001764',
'GO:0036513',
'GO:0032757',
'GO:0007026',
'GO:0060373',
'GO:0010181',
'GO:0004601',
'GO:0060394',
'GO:0035166',
'GO:0016301',
'GO:0021740',
'GO:0001503',
'GO:0042407',
'GO:0008013',
'GO:0001542',
'GO:0008144',
'GO:0007283',
'GO:0046034',
'GO:0016579',
'GO:0035970',
'GO:1990184',
'GO:0072179',
'GO:0030219',
'GO:0061113',
'GO:0009954',
'GO:0031667',
'GO:0060527',
'GO:0010717',
'GO:0048856',
'GO:0005782',
'GO:0016339',
'GO:0048146',
'GO:0006611',
'GO:0043005',
'GO:0043278',
'GO:0051726',
'GO:0004707',
'GO:0035976',
'GO:0098919',
'GO:0031124',
'GO:0010499',
'GO:1901215',
'GO:0005819',
'GO:0034612',
'GO:0043161',
'GO:0035640',
'GO:0005640',
'GO:0051918',
'GO:0045899',
'GO:0070491',
'GO:0032434',
'GO:0016558',
'GO:0048167',
'GO:0007494',
'GO:0070093',
'GO:0055074',
'GO:0017144',
'GO:0090630',
'GO:0033065',
'GO:0070715',
'GO:0001972',
'GO:0060048',
'GO:0035064',
'GO:2000194',
'GO:0006892',
'GO:0033613',
'GO:0021854',
'GO:0034378',
'GO:0102752',
'GO:0052746',
'GO:0014070',
'GO:0030858',
'GO:0097291',
'GO:0032695',
'GO:0008138',
'GO:0006887',
'GO:0048667',
'GO:0048812',
'GO:0005251',
'GO:0006171',
'GO:0019107',
'GO:1902473',
'GO:0015910',
'GO:2000573',
'GO:0034695',
'GO:0060425',
'GO:0046888',
'GO:0033120',
'GO:0015824',
'GO:0045747',
'GO:0099576',
'GO:0042605',
'GO:0001578',
'GO:0006363',
'GO:0045078',
'GO:0016236',
'GO:0004849',
'GO:1902882',
'GO:0030432',
'GO:0032454',
'GO:0008271',
'GO:0032590',
'GO:0002063',
'GO:0042742',
'GO:0035767',
'GO:0070125',
'GO:0045766',
'GO:0008134',
'GO:0098685',
'GO:0004055',
'GO:0060231',
'GO:0019899',
'GO:0048041',
'GO:0071339',
'GO:0031904',
'GO:0008284',
'GO:0007420',
'GO:0033033',
'GO:0047555',
'GO:0039694',
'GO:0043928',
'GO:0097178',
'GO:0072686',
'GO:0070306',
'GO:0046620',
'GO:0071944',
'GO:0035696',
'GO:0003697',
'GO:0004843',
'GO:0070286',
'GO:0006543',
'GO:0060710',
'GO:0010508',
'GO:0010629',
'GO:0005344',
'GO:0050923',
'GO:0006636',
'GO:0015501',
'GO:0002232',
'GO:0031434',
'GO:0070268',
'GO:0006616',
'GO:0072657',
'GO:0050839',
'GO:0032675',
'GO:0044291',
'GO:0008024',
'GO:0014740',
'GO:0045727',
'GO:0031072',
'GO:0006481',
'GO:0055085',
'GO:0030676',
'GO:0043154',
'GO:0004735',
'GO:0000182',
'GO:0032099',
'GO:0005667',
'GO:0032570',
'GO:0030277',
'GO:2000683',
'GO:0048256',
'GO:0031091',
'GO:0045669',
'GO:0070050',
'GO:0090136',
'GO:0072050',
'GO:0006071',
'GO:0098656',
'GO:0008045',
'GO:0007346',
'GO:0032585',
'GO:0035264',
'GO:0045095',
'GO:0002062',
'GO:0071816',
'GO:0031670',
'GO:2000576',
'GO:0032588',
'GO:0002218',
'GO:0048341',
'GO:0021555',
'GO:0007288',
'GO:0007608',
'GO:1904837',
'GO:0015804',
'GO:2000646',
'GO:0035861',
'GO:0015226',
'GO:0002125',
'GO:0002020',
'GO:1902036',
'GO:0007043',
'GO:0009922',
'GO:0003151',
'GO:0003823',
'GO:0006888',
'GO:0051491',
'GO:0016342',
'GO:0002322',
'GO:0019725',
'GO:0072300',
'GO:0034767',
'GO:0030838',
'GO:0001883',
'GO:0097200',
'GO:0051013',
'GO:0032782',
'GO:0015035',
'GO:0060421',
'GO:1905450',
'GO:0006368',
'GO:0000979',
'GO:1990138',
'GO:0042308',
'GO:0051434',
'GO:0043231',
'GO:0060022',
'GO:0032940',
'GO:0002221',
'GO:0030877',
'GO:0072511',
'GO:0008021',
'GO:1902043',
'GO:1990841',
'GO:0001517',
'GO:0005902',
'GO:0032289',
'GO:0016607',
'GO:0010884',
'GO:0042214',
'GO:0043583',
'GO:0042327',
'GO:0001953',
'GO:0019903',
'GO:0016242',
'GO:0019933',
'GO:0035060',
'GO:2000553',
'GO:0002155',
'GO:0051262',
'GO:0006468',
'GO:0071013',
'GO:0043388',
'GO:0035624',
'GO:0045786',
'GO:0035327',
'GO:1902750',
'GO:0030331',
'GO:0008094',
'GO:0072040',
'GO:0050196',
'GO:0010762',
'GO:0070051',
'GO:0098982',
'GO:0021747',
'GO:0070256',
'GO:0043568',
'GO:0055093',
'GO:0072359',
'GO:0030705',
'GO:0004724',
'GO:0019370',
'GO:0045214',
'GO:0051291',
'GO:1990090',
'GO:0015884',
'GO:0019086',
'GO:0010634',
'GO:0006537',
'GO:0010510',
'GO:0002439',
'GO:0003333',
'GO:0032233',
'GO:0035025',
'GO:0002082',
'GO:0012505',
'GO:0032212',
'GO:0043065',
'GO:1990356',
'GO:0042288',
'GO:0035162',
'GO:0001649',
'GO:0002639',
'GO:0034274',
'GO:0017048',
'GO:0005929',
'GO:0007613',
'GO:0030374',
'GO:0030550',
'GO:0007507',
'GO:0004708',
'GO:0017147',
'GO:0098981',
'GO:0050965',
'GO:0048365',
'GO:0015106',
'GO:0031077',
'GO:0008553',
'GO:0017158',
'GO:0060351',
'GO:0038066',
'GO:0043001',
'GO:0045087',
'GO:1902722',
'GO:0045069',
'GO:0015695',
'GO:0030097',
'GO:0033627',
'GO:0002327',
'GO:0005160',
'GO:0051444',
'GO:1904985',
'GO:0006556',
'GO:0008543',
'GO:0015888',
'GO:0006734',
'GO:0010165',
'GO:0043204',
'GO:0030325',
'GO:0060272',
'GO:0030301',
'GO:0008622',
'GO:0005314',
'GO:0005105',
'GO:1901016',
'GO:0042162',
'GO:0000050',
'GO:0008294',
'GO:0006956',
'GO:0010976',
'GO:2000320',
'GO:0002223',
'GO:0035855',
'GO:0032467',
'GO:0006099',
'GO:0043496',
'GO:0001085',
'GO:0045995',
'GO:0007265',
'GO:0030315',
'GO:0010750',
'GO:0001223',
'GO:0007631',
'GO:0004430',
'GO:0042249',
'GO:0005499',
'GO:0051019',
'GO:1990809',
'GO:0015193',
'GO:0031098',
'GO:0048468',
'GO:0002686',
'GO:0060745',
'GO:0009953',
'GO:0099055',
'GO:0031396',
'GO:0015803',
'GO:0034040',
'GO:0006596',
'GO:0004523',
'GO:0007158',
'GO:0004704',
'GO:0072583',
'GO:0060345',
'GO:1902936',
'GO:0031293',
'GO:1903441',
'GO:0030517',
'GO:0090156',
'GO:0019776',
'GO:0045967',
'GO:0048729',
'GO:0005840',
'GO:0070487',
'GO:1901653',
'GO:0009749',
'GO:0060413',
'GO:0005585',
'GO:0005070',
'GO:2001259',
'GO:0072141',
'GO:0048661',
'GO:0009617',
'GO:0008356',
'GO:0006696',
'GO:0019674',
'GO:0045472',
'GO:1901841',
'GO:0050729',
'GO:0005085',
'GO:0045893',
'GO:0007597',
'GO:0010039',
'GO:0000578',
'GO:0030324',
'GO:0004198',
'GO:0000791',
'GO:0004842',
'GO:0016028',
'GO:0006656',
'GO:0016298',
'GO:0051099',
'GO:0008608',
'GO:0015093',
'GO:0006695',
'GO:0044331',
'GO:0042110',
'GO:0004796',
'GO:0015301',
'GO:0006120',
'GO:0001707',
'GO:0005229',
'GO:0015721',
'GO:0047485',
'GO:0043069',
'GO:0001692',
'GO:0035473',
'GO:0048096',
'GO:0006457',
'GO:0090132',
'GO:0000480',
'GO:0030246',
'GO:0001782',
'GO:0015696',
'GO:0030054',
'GO:0006112',
'GO:0050921',
'GO:0035723',
'GO:0070644',
'GO:0010812',
'GO:0033138',
'GO:0042359',
'GO:0051683',
'GO:1902358',
'GO:0061629',
'GO:0043403',
'GO:0004497',
'GO:0014054',
'GO:0045665',
'GO:0006952',
'GO:0031674',
'GO:0070776',
'GO:0001771',
'GO:0035902',
'GO:0036505',
'GO:0032925',
'GO:0008283',
'GO:0090503',
'GO:0015697',
'GO:0005901',
'GO:0071787',
'GO:0010824',
'GO:0006401',
'GO:0010875',
'GO:0006516',
'GO:0030853',
'GO:0032793',
'GO:1902459',
'GO:0061290',
'GO:0005089',
'GO:0008285',
'GO:0035255',
'GO:0051088',
'GO:0070063',
'GO:0006979',
'GO:1902045',
'GO:0010611',
'GO:0032024',
'GO:0006499',
'GO:0003266',
'GO:0034765',
'GO:0030126',
'GO:0048536',
'GO:0045595',
'GO:1900028',
'GO:0032795',
'GO:0009330',
'GO:0042416',
'GO:0018963',
'GO:0008154',
'GO:0017080',
'GO:0040016',
'GO:0090267',
'GO:0007094',
'GO:0043235',
'GO:0001958',
'GO:0046872',
'GO:2000969',
'GO:0043631',
'GO:0090096',
'GO:0005753',
'GO:0016021',
'GO:0045599',
'GO:0030594',
'GO:0150076',
'GO:0031032',
'GO:0006911',
'GO:1903988',
'GO:0048701',
'GO:0033137',
'GO:0006693',
'GO:0046933',
'GO:0001945',
'GO:0050859',
'GO:0006940',
'GO:1903237',
'GO:0001806',
'GO:0045178',
'GO:0005719',
'GO:0001525',
'GO:0061665',
'GO:0008104',
'GO:0004090',
'GO:0019226',
'GO:2000504',
'GO:0044030',
'GO:2000546',
'GO:0044297',
'GO:0005981',
'GO:0072092',
'GO:0045070',
'GO:0050795',
'GO:0046579',
'GO:0060341',
'GO:0005887',
'GO:0030318',
'GO:0000123',
'GO:0002039',
'GO:0048841',
'GO:0060441',
'GO:0030890',
'GO:0010523',
'GO:0043243',
'GO:0099562',
'GO:0001736',
'GO:0032026',
'GO:0032880',
'GO:0001725',
'GO:0033353',
'GO:0017081',
'GO:0031430',
'GO:0070852',
'GO:0010818',
'GO:0060044',
'GO:0010569',
'GO:0005791',
'GO:0072562',
'GO:0007270',
'GO:0005793',
'GO:0000783',
'GO:0016471',
'GO:0004407',
'GO:0010944',
'GO:0071481',
'GO:0032094',
'GO:0071364',
'GO:0035374',
'GO:0060068',
'GO:0016525',
'GO:0051591',
'GO:0043010',
'GO:0006470',
'GO:1905337',
'GO:0034374',
'GO:0007274',
'GO:0045994',
'GO:0045111',
'GO:0043202',
'GO:0007417',
'GO:0045737',
'GO:0060510',
'GO:0021759',
'GO:0042756',
'GO:0006378',
'GO:0043194',
'GO:0010748',
'GO:0043548',
'GO:0071418',
'GO:0033344',
'GO:0016239',
'GO:0030145',
'GO:0006298',
'GO:1902894',
'GO:0008076',
'GO:1902600',
'GO:0060717',
'GO:0098664',
'GO:0021915',
'GO:0019805',
'GO:0050885',
'GO:0031694',
'GO:0002318',
'GO:0007605',
'GO:0004703',
'GO:0071679',
'GO:0009792',
'GO:0007179',
'GO:0098562',
'GO:0003356',
'GO:0008234',
'GO:0042493',
'GO:1902261',
'GO:0043488',
'GO:0007166',
'GO:1904349',
'GO:0016241',
'GO:0005048',
'GO:0007254',
'GO:0008327',
'GO:0031023',
'GO:0045162',
'GO:0043277',
'GO:0051525',
'GO:0031093',
'GO:0042953',
'GO:0038084',
'GO:0071773',
'GO:0042135',
'GO:0050870',
'GO:0016032',
'GO:0060689',
'GO:0006369',
'GO:0008593',
'GO:0002581',
'GO:0015101',
'GO:0010608',
'GO:0045171',
'GO:0045785',
'GO:0007156',
'GO:0060628',
'GO:0015464',
'GO:0048791',
'GO:0031325',
'GO:0005750',
'GO:0032400',
'GO:0051026',
'GO:0060761',
'GO:0019003',
'GO:0035580',
'GO:0001523',
'GO:0031083',
'GO:0001891',
'GO:0044729',
'GO:1903697',
'GO:0019905',
'GO:0015734',
'GO:0035924',
'GO:0015808',
'GO:2000974',
'GO:0045742',
'GO:0042542',
'GO:0035529',
'GO:0033572',
'GO:0101003',
'GO:0006754',
'GO:0030574',
'GO:0003954',
'GO:0016807',
'GO:2000791',
'GO:0071880',
'GO:1900103',
'GO:0060122']

In [70]:
listsofGO[0]

'GO:2000377'

In [87]:
goDATA['GO:0035249'].name

'synaptic transmission, glutamatergic'

In [75]:
for go in listsofGO:
    str1 = goDATA[go].name
    str2 = goDATA[go].namespace
    

AttributeError: 'str' object has no attribute 'name'